In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130963 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive  -o nonempty
##重启kernel时安装的gcloud

In [0]:
!ls

adc.json  drive  sample_data


In [0]:
import os
os.chdir('drive/mobile_adv/code')

In [0]:
# Install TensorFlow gpu version
!pip install -q tensorflow-gpu==2.0.0-beta1
import tensorflow as tf
print("GPU Available: ", tf.test.is_gpu_available())
print(tf.__version__)

# # Install TensorFlow cpu version
# !pip install tensorflow==2.0.0-beta1
# import tensorflow as tf
# print(tf.__version__)

     |████████████████████████████████| 348.9MB 93kB/s 
     |████████████████████████████████| 3.1MB 29.6MB/s 
     |████████████████████████████████| 501kB 22.6MB/s 
GPU Available:  True
1.14.0


In [0]:
root_path='../data'

In [0]:
train_path=root_path+'/round1_iflyad_anticheat_traindata.txt'
test_path=root_path+'/round1_iflyad_anticheat_testdata_feature.txt'
submission_path=root_path+'/营销反欺诈-sample.csv'

In [0]:
import pandas as pd

In [0]:
train=pd.read_csv(train_path,delimiter='\t')
test=pd.read_csv(test_path,delimiter='\t')
submission=pd.read_csv(submission_path)

In [0]:
train.info()

NameError: ignored

In [0]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
sid             100000 non-null object
pkgname         100000 non-null object
ver             70808 non-null object
adunitshowid    100000 non-null object
mediashowid     100000 non-null object
apptype         100000 non-null float64
nginxtime       100000 non-null float64
ip              100000 non-null object
city            98533 non-null object
province        100000 non-null float64
reqrealip       100000 non-null object
adidmd5         100000 non-null object
imeimd5         100000 non-null object
idfamd5         100000 non-null object
openudidmd5     100000 non-null object
macmd5          100000 non-null object
dvctype         100000 non-null float64
model           99905 non-null object
make            88496 non-null object
ntt             100000 non-null float64
carrier         100000 non-null float64
os              100000 non-null object
osv             99068 non-null

In [0]:
train.head()

,sid,label,pkgname,ver,adunitshowid,mediashowid,apptype,nginxtime,ip,city,province,reqrealip,adidmd5,imeimd5,idfamd5,openudidmd5,macmd5,dvctype,model,make,ntt,carrier,os,osv,orientation,lan,h,w,ppi
0,d7460126-e071-4979-9ee8-42f72777a28a-156009070...,1,2d2664e827bcbb8b07100c7fbe072e9b,11.11.0,907d0f8c29663840491577a21c7b612a,ca64a500000d84c8fcb8a0587d0e1e0c,280.0,1.560091e+12,183.197.47.83,石家庄市,-1.0,180.76.181.7,eb4fa0cf77cdd57a6993a4e42c8fa4cf,930b3a8ecff4f586c63fec5b96693f32,empty,empty,ab96caaba2b7973e63fb0d3d5c5bf8b0,2.0,V1809A,vivo,2.0,46000.0,Android,8.1.0,0.0,zh-CN,2340.0,1080.0,3.0
1,b660d559-db97-4b5f-9bd2-2450cb89ce77-156005074...,1,empty,NaN,10199dc8fea2e173525bc3151bd84312,3f2fc57a6e1f9c6fa4464c25cc1e88a3,319.0,1.560051e+12,106.34.14.149,开封市,-1.0,47.95.111.138,a4ae6efcfaee62fb6da44a9c05753259,6e38deaa1f7b4118015c550bb21913d2,empty,a4ae6efcfaee62fb6da44a9c05753259,0f607264fc6318a92b9e13c65db7cd3c,2.0,BLA-AL00,HUAWEI,5.0,0.0,android,Android_9,0.0,NaN,1080.0,2040.0,0.0
2,f49a740e-66c3-4605-9b67-4d3079fe69cb-156008914...,0,16b81f93f4b1a35cebbf15f07683f171,3.2.1.0524.1958,83f2ecfe65f936f5f2ed59f8e8ff1d01,eea7280e1a2313e4c2e89290b01d196c,273.0,1.560089e+12,223.104.16.151,长春市,-1.0,39.96.159.155,3404f799628be2cf8fda7428aac2cca4,3fa42b06332c729f530b973973fabe3d,empty,empty,5ba1cf1efeee4f83e91f9c0adc3015e3,2.0,PBEM00,OPPO,2.0,46000.0,Android,8.1.0,1.0,zh-CN,2196.0,1080.0,0.0
3,fd60d096-f168-4540-b782-729d64d0fcc6-156006253...,0,empty,4.7.5,9f1eadd9092b19bc86ee0cacde1c867f,eec946a5a66c023ec9d3b2ede5900626,265.0,1.560063e+12,223.104.239.101,曲靖市,-1.0,47.101.65.230,941517d46b7261d98592425672bb86f5,04420edf96dde14c357db3e83d0b2310,empty,empty,empty,2.0,M711C,NaN,6.0,0.0,android,7.0,0.0,NaN,0.0,0.0,0.0
4,a037b032-a5c7-40ea-9161-26b118b12406-156007938...,1,cf4821986014a7fef9d7b7ad8de655e4,228,2af944462e43cd2f59acbbfd37445413,57b3053174973702549ba88b6017ac30,336.0,1.560079e+12,220.152.155.170,深圳市,-1.0,39.96.183.23,fc08a6724db4c2dcf9b5af3a4e0b6f38,23e94b65bab381bf57169149a20c0520,empty,empty,4363320019cbb7cde8b704908b673416,2.0,HUAWEI EML-AL00,EML-AL00,2.0,46000.0,Android,9,0.0,Zh-CN,2244.0,1080.0,0.0


In [0]:
test.head()


,sid,pkgname,ver,adunitshowid,mediashowid,apptype,nginxtime,ip,city,province,reqrealip,adidmd5,imeimd5,idfamd5,openudidmd5,macmd5,dvctype,model,make,ntt,carrier,os,osv,orientation,lan,h,w,ppi
0,4601f528-865c-4238-a43f-cf4a4e11027d-156017418...,empty,190516,51f02f6007e49301adb4ccb1da0fbf12,5f83d3d963b19352aff3d9b5b29fa736,273.0,1.560174e+12,117.136.86.37,西安市,-1.0,120.132.53.176,9d87e5b432ac8f3048cf6ee38c867cfe,8be7e89c94848c14020a77aa175f6dbb,empty,empty,52cf13a05c72f6bd9167d622be314665,3.0,JKM-AL00,HUAWEI,6.0,46000.0,android,9,0.0,zh_CN,2137.0,1080.0,0.0
1,c727e4bf-2ae1-4829-9aa1-730277ee2042-156013863...,empty,4.2.0,96b536f3322df1404342c933fe597569,4d24206fcbc095ad0c27c7dd10e83047,318.0,1.560139e+12,110.52.34.129,株洲市,-1.0,139.199.129.29,3c24b405e9d0c055da356bbe991c4ded,5e5dd48daa023b3577269289faba142c,empty,empty,8396426c76002707ba8dde10297f3525,2.0,OPPO R9S PLUS,OPPO,2.0,46003.0,android,6.0,0.0,NaN,1080.0,1920.0,480.0
2,fe23d40a-0db8-403f-8df0-6e8b9ea461c3-156009638...,ae90f961dbafe0f78f8a1cd7775362ed,NaN,0d116d0416862dea0ff81905de4df513,fff9eaef85a611570dda886b390fee8a,281.0,1.560096e+12,120.229.25.233,深圳市,-1.0,123.123.219.25,7c0f4ff62a773520c11b5eb6dfba2d74,empty,empty,empty,empty,2.0,HUAWEI MLA-AL10,HUAWEI,2.0,-1.0,android,3.9.0,0.0,NaN,0.0,0.0,0.0
3,308b053c-5d7c-43b8-99dc-8aad647f953f-156017652...,empty,190516,51f02f6007e49301adb4ccb1da0fbf12,5f83d3d963b19352aff3d9b5b29fa736,273.0,1.560177e+12,117.136.19.239,无锡市,-1.0,117.50.25.7,f1604500eddb56dbd52d2a2ce2ca17e7,fa66fe6b853f069102faf3a55556102f,empty,empty,15cb635f22476ff6640f9f5f20262da8,3.0,PACT00,OPPO,6.0,46000.0,android,8.1.0,0.0,zh_CN,2280.0,1080.0,0.0
4,94477644-63cd-4f3c-9b2a-f3d25f1ec7e2-156014459...,170a88a12e36f8a0f1b73442304398b1,30927000,6618408bae888ad02c4e0d9bd4d88d29,d53d2af198ebef9544f0823c3c8e84f8,301.0,1.560145e+12,218.89.52.71,南充市,-1.0,39.96.159.161,empty,d7bdea1dc04a9cc31d180e40ea9b6110,empty,empty,5ba1cf1efeee4f83e91f9c0adc3015e3,2.0,OC105,SMARTISAN,2.0,46000.0,Android,7.1.2,1.0,zh-CN,720.0,360.0,0.0


In [0]:
train=train.append(test,ignore_index=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [0]:
#########################################################################

In [0]:
import time
train['day'] = train['nginxtime'].apply(lambda x : int(time.strftime("%d", time.localtime(x))))
train['hour'] = train['nginxtime'].apply(lambda x : int(time.strftime("%H", time.localtime(x))))


In [0]:
print(sorted(train['day'].unique()))
print(sorted(train['hour'].unique()))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [0]:
train.loc[train['label'].isnull(),['day']].groupby('day').size()

day
1     3261
2     3312
3     3260
4     3337
5     3279
6     3381
7     3363
8     3315
9     3258
10    3302
11    3237
12    3395
13    3292
14    3223
15    3233
16    3353
17    3343
18    3272
19    3285
20    3192
21    3152
22    3285
23    3244
24    3260
25    3272
26    3329
27    3240
28    3358
29    3073
30    3035
31    1859
dtype: int64

In [0]:
train.loc[train['label'].isnull(),['day','label']].drop_duplicates().sort_values(by='day')

In [0]:
train.head()

,adidmd5,adunitshowid,apptype,carrier,city,dvctype,h,idfamd5,imeimd5,ip,label,lan,macmd5,make,mediashowid,model,nginxtime,ntt,openudidmd5,orientation,os,osv,pkgname,ppi,province,reqrealip,sid,ver,w,day,hour
0,eb4fa0cf77cdd57a6993a4e42c8fa4cf,907d0f8c29663840491577a21c7b612a,280.0,46000.0,石家庄市,2.0,2340.0,empty,930b3a8ecff4f586c63fec5b96693f32,183.197.47.83,1.0,zh-CN,ab96caaba2b7973e63fb0d3d5c5bf8b0,vivo,ca64a500000d84c8fcb8a0587d0e1e0c,V1809A,1.560091e+12,2.0,empty,0.0,Android,8.1.0,2d2664e827bcbb8b07100c7fbe072e9b,3.0,-1.0,180.76.181.7,d7460126-e071-4979-9ee8-42f72777a28a-156009070...,11.11.0,1080.0,23,8
1,a4ae6efcfaee62fb6da44a9c05753259,10199dc8fea2e173525bc3151bd84312,319.0,0.0,开封市,2.0,1080.0,empty,6e38deaa1f7b4118015c550bb21913d2,106.34.14.149,1.0,NaN,0f607264fc6318a92b9e13c65db7cd3c,HUAWEI,3f2fc57a6e1f9c6fa4464c25cc1e88a3,BLA-AL00,1.560051e+12,5.0,a4ae6efcfaee62fb6da44a9c05753259,0.0,android,Android_9,empty,0.0,-1.0,47.95.111.138,b660d559-db97-4b5f-9bd2-2450cb89ce77-156005074...,NaN,2040.0,15,19
2,3404f799628be2cf8fda7428aac2cca4,83f2ecfe65f936f5f2ed59f8e8ff1d01,273.0,46000.0,长春市,2.0,2196.0,empty,3fa42b06332c729f530b973973fabe3d,223.104.16.151,0.0,zh-CN,5ba1cf1efeee4f83e91f9c0adc3015e3,OPPO,eea7280e1a2313e4c2e89290b01d196c,PBEM00,1.560089e+12,2.0,empty,1.0,Android,8.1.0,16b81f93f4b1a35cebbf15f07683f171,0.0,-1.0,39.96.159.155,f49a740e-66c3-4605-9b67-4d3079fe69cb-156008914...,3.2.1.0524.1958,1080.0,5,7
3,941517d46b7261d98592425672bb86f5,9f1eadd9092b19bc86ee0cacde1c867f,265.0,0.0,曲靖市,2.0,0.0,empty,04420edf96dde14c357db3e83d0b2310,223.104.239.101,0.0,NaN,empty,NaN,eec946a5a66c023ec9d3b2ede5900626,M711C,1.560063e+12,6.0,empty,0.0,android,7.0,empty,0.0,-1.0,47.101.65.230,fd60d096-f168-4540-b782-729d64d0fcc6-156006253...,4.7.5,0.0,1,9
4,fc08a6724db4c2dcf9b5af3a4e0b6f38,2af944462e43cd2f59acbbfd37445413,336.0,46000.0,深圳市,2.0,2244.0,empty,23e94b65bab381bf57169149a20c0520,220.152.155.170,1.0,Zh-CN,4363320019cbb7cde8b704908b673416,EML-AL00,57b3053174973702549ba88b6017ac30,HUAWEI EML-AL00,1.560079e+12,2.0,empty,0.0,Android,9,cf4821986014a7fef9d7b7ad8de655e4,0.0,-1.0,39.96.183.23,a037b032-a5c7-40ea-9161-26b118b12406-156007938...,228,1080.0,13,7


In [0]:
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1100000 entries, 0 to 99999
Data columns (total 29 columns):
adidmd5         1100000 non-null object
adunitshowid    1100000 non-null object
apptype         1100000 non-null float64
carrier         1100000 non-null float64
city            1083340 non-null object
dvctype         1100000 non-null float64
h               1100000 non-null float64
idfamd5         1100000 non-null object
imeimd5         1100000 non-null object
ip              1100000 non-null object
label           1000000 non-null float64
lan             699138 non-null object
macmd5          1100000 non-null object
make            964657 non-null object
mediashowid     1100000 non-null object
model           1095438 non-null object
nginxtime       1100000 non-null float64
ntt             1100000 non-null float64
openudidmd5     1100000 non-null object
orientation     1100000 non-null float64
os              1100000 non-null object
osv             1086071 non-null object
pkg

In [0]:
import pandas as pd

def null_ratio(a):
  data=pd.DataFrame()

  dd=[]
  cc=[]
  rr=[]

  for i in a.columns:
    d=len(a)-a[i].count()
    r=(d/len(a))*100
    rate='%.2f%%' % r
    print('字段名为：',str(i).ljust(10),'缺失值数量:',str(d).ljust(4),'缺失数量占比：',rate) #这里print主要是为了在脚本中观察是否获取到想要的数据，方便调试。
    dd.append(i)
    cc.append(d)
    rr.append(rate)

  data[u'字段名为']=dd
  data[u'缺失值数量']=cc
  data[u'缺失数量占比']=rr
  data=data.sort_values(by='缺失数量占比',ascending=False)
  return data

In [0]:
# def unique_amout(df):
  

In [0]:
df_null_ratio=null_ratio(train)

字段名为： adidmd5    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： adunitshowid 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： apptype    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： carrier    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： city       缺失值数量: 16660 缺失数量占比： 1.51%
字段名为： dvctype    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： h          缺失值数量: 0    缺失数量占比： 0.00%
字段名为： idfamd5    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： imeimd5    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： ip         缺失值数量: 0    缺失数量占比： 0.00%
字段名为： label      缺失值数量: 100000 缺失数量占比： 9.09%
字段名为： lan        缺失值数量: 400862 缺失数量占比： 36.44%
字段名为： macmd5     缺失值数量: 0    缺失数量占比： 0.00%
字段名为： make       缺失值数量: 135343 缺失数量占比： 12.30%
字段名为： mediashowid 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： model      缺失值数量: 4562 缺失数量占比： 0.41%
字段名为： nginxtime  缺失值数量: 0    缺失数量占比： 0.00%
字段名为： ntt        缺失值数量: 0    缺失数量占比： 0.00%
字段名为： openudidmd5 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： orientation 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： os         缺失值数量: 0    缺失数量占比： 0.00%
字段名为： osv        缺失值数量: 13929 缺失数量占比： 1.27%
字段名为： pkgname    缺失值数量: 0    缺失数量占比： 0.

In [0]:
df_null_ratio

,字段名为,缺失值数量,缺失数量占比
10,label,100000,9.09%
11,lan,400862,36.44%
27,ver,351016,31.91%
13,make,135343,12.30%
4,city,16660,1.51%
21,osv,13929,1.27%
15,model,4562,0.41%
0,adidmd5,0,0.00%
19,orientation,0,0.00%
20,os,0,0.00%


In [0]:

encoder=['pkgname','ver','adunitshowid','mediashowid','ip','city','province','reqrealip','adidmd5','imeimd5',
              'idfamd5','openudidmd5','macmd5','model','make','carrier','os','lan','osv',
             'apptype','dvctype','ntt','orientation']
numerical_var=['nginxtime','h','w','ppi']

In [0]:
from sklearn.preprocessing import LabelEncoder
col_encoder = LabelEncoder()
for feat in encoder:
    col_encoder.fit(train[feat].astype(str))
    train[feat] = col_encoder.transform(train[feat].astype(str))

In [0]:
train.sample(10)

,adidmd5,adunitshowid,apptype,carrier,city,dvctype,h,idfamd5,imeimd5,ip,label,lan,macmd5,make,mediashowid,model,nginxtime,ntt,openudidmd5,orientation,os,osv,pkgname,ppi,province,reqrealip,sid,ver,w
3472,731103,620,70,2,236,0,0.0,336,540912,318064,0.0,19,308687,2307,6,6747,1.560088e+12,2,79726,0,1,139,1461,0.0,0,3747,a75b7de2-5ff6-4b5b-abdb-d83417c9bc9d-156008817...,3266,0.0
796141,731103,778,50,2,164,1,760.0,336,433901,345296,1.0,24,308687,2307,255,6734,1.559629e+12,2,79726,1,0,139,235,0.0,0,8669,7b535600-93b3-4fd0-84df-2b1efa2de77a-155962877...,1802,360.0
31672,731103,770,50,2,274,1,760.0,336,338341,582458,1.0,24,308687,2307,255,6734,1.560033e+12,5,79726,1,0,139,235,0.0,0,8665,10e4e62a-0f72-4e4d-ade3-ded96c4cc1b2-156003339...,1830,360.0
718977,499594,610,64,4,195,1,2310.0,336,457784,552238,1.0,24,19886,476,35,4984,1.559688e+12,6,79726,0,1,140,2214,0.0,0,8889,8fefafd8-3142-4f0e-9ba6-41c38c96bd0a-155968812...,2404,1080.0
507301,731103,339,50,2,84,1,760.0,336,450147,80439,1.0,24,117628,1032,255,4940,1.559814e+12,2,79726,1,0,139,235,0.0,2,8667,b94c45ff-0367-4ae5-be80-92f4316d4ddb-155981379...,1830,360.0
582700,283739,610,64,3,297,1,2244.0,336,250394,68835,1.0,24,19886,492,35,1949,1.559751e+12,2,79726,0,1,140,2214,0.0,0,8885,eff9409c-d77f-4b1b-b6c0-ca3778e068e1-155975081...,2404,1080.0
31248,353686,249,51,2,157,1,1280.0,336,71319,406717,NaN,22,292364,492,170,637,1.560098e+12,2,79726,0,1,134,1506,480.0,0,3837,f407667a-335f-48b6-93f9-a533ac917344-156009757...,2798,720.0
711733,485967,444,60,2,123,1,1806.0,336,822824,531646,1.0,22,19886,492,258,1211,1.559733e+12,2,79726,0,0,140,1222,3.0,0,520,ca93a401-cc63-42b4-9f29-f4f6e3adf42d-155973252...,526,1080.0
672814,731103,339,50,2,259,1,760.0,336,580600,254175,1.0,24,117628,2307,255,6394,1.559732e+12,5,79726,1,0,139,235,0.0,0,8666,8ffd8cbc-13c3-494d-8130-9e870e45b7bd-155973174...,1801,360.0
570154,731103,778,50,2,259,1,864.0,336,758260,377245,0.0,24,308687,1512,255,4851,1.559799e+12,5,79726,1,0,101,235,0.0,0,8663,59c38073-4c86-46f0-959e-c0e8840204df-155979865...,1801,432.0


In [0]:
##直接用一个
nuni_var=train.nunique()

In [0]:
type(nuni_var)
print(nuni_var.index)
dd=[each for each in zip(nuni_var.index,nuni_var.values)]
print(sorted(dd,key=lambda item_tuple:item_tuple[1] ))
# sd = sorted(data,key=lambda item:item[1],reverse=False)

Index(['adidmd5', 'adunitshowid', 'apptype', 'carrier', 'city', 'dvctype', 'h',
       'idfamd5', 'imeimd5', 'ip', 'label', 'lan', 'macmd5', 'make',
       'mediashowid', 'model', 'nginxtime', 'ntt', 'openudidmd5',
       'orientation', 'os', 'osv', 'pkgname', 'ppi', 'province', 'reqrealip',
       'sid', 'ver', 'w', 'day', 'hour'],
      dtype='object')
[('label', 2), ('os', 2), ('dvctype', 3), ('orientation', 4), ('carrier', 5), ('ntt', 8), ('province', 8), ('hour', 24), ('day', 31), ('lan', 34), ('apptype', 91), ('ppi', 119), ('osv', 186), ('mediashowid', 313), ('city', 332), ('idfamd5', 360), ('w', 449), ('adunitshowid', 800), ('h', 985), ('pkgname', 2368), ('make', 2728), ('ver', 3269), ('model', 7958), ('reqrealip', 9748), ('openudidmd5', 85051), ('macmd5', 329184), ('adidmd5', 780369), ('ip', 813719), ('imeimd5', 1021836), ('nginxtime', 1098977), ('sid', 1100000)]


In [0]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder()
one_hot_var=['carrier','dvctype','orientation','os']
feat_one_hot_sparse_matrix=enc.fit_transform(train[one_hot_var])

    
# for feature in cate_feature:
#     enc.fit(data[feature].values.reshape(-1, 1))
#     base_train_csr = sparse.hstack((base_train_csr, enc.transform(train_x[feature].values.reshape(-1, 1))), 'csr',
#                                    'bool')
#     base_predict_csr = sparse.hstack((base_predict_csr, enc.transform(predict[feature].values.reshape(-1, 1))),
#                                      'csr',
#                                      'bool')
print('one-hot prepared !')


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


one-hot prepared !


In [0]:
print(feat_one_hot_sparse_matrix[0][0])
print(feat_one_hot_sparse_matrix.shape)

  (0, 2)	1.0
  (0, 6)	1.0
  (0, 8)	1.0
  (0, 12)	1.0
(1100000, 14)


In [0]:
train.shape

(1100000, 31)

In [0]:
#应该有简化方法，后面在改如value_counts
# large_base_var里要去掉数值变量
large_base_var=['adidmd5', 'adunitshowid', 'apptype', 'city', 'idfamd5',
 'imeimd5', 'ip', 'lan', 'macmd5', 'make', 'mediashowid', 'model', 
 'ntt', 'openudidmd5', 'osv', 'pkgname',
 'province', 'reqrealip', 'sid', 'ver']

# numerical_var=['nginxtime','h','w','ppi']
for each in large_base_var:
  df_temp=train.groupby([each])[each].count().to_frame().rename(columns={each:'{}_count'.format(each)}).reset_index()
  train=pd.merge(train,df_temp,on=[each],how='left')
#   print(df_temp.head(20))
#   break

  








In [0]:
print(train.shape)
train.head(20)

(1100000, 51)


,adidmd5,adunitshowid,apptype,carrier,city,dvctype,h,idfamd5,imeimd5,ip,label,lan,macmd5,make,mediashowid,model,nginxtime,ntt,openudidmd5,orientation,os,osv,pkgname,ppi,province,reqrealip,sid,ver,w,day,hour,adidmd5_count,adunitshowid_count,apptype_count,city_count,idfamd5_count,imeimd5_count,ip_count,lan_count,macmd5_count,make_count,mediashowid_count,model_count,ntt_count,openudidmd5_count,osv_count,pkgname_count,province_count,reqrealip_count,sid_count,ver_count
0,717033,460,41,2,229,1,2340.0,336,587277,518048,1.0,24,220316,2307,243,6397,1.560091e+12,2,79726,0,0,139,409,3.0,0,5588,d7460126-e071-4979-9ee8-42f72777a28a-156009070...,182,1080.0,23,8,1,3814,15888,12370,1099640,1,1,539581,1,226658,15888,9492,698700,1011615,334610,15888,1001057,22584,1,11286
1,502204,46,61,1,119,1,1080.0,336,440657,45216,1.0,19,19886,492,62,622,1.560051e+12,5,54847,0,1,178,2214,0.0,0,8934,b660d559-db97-4b5f-9bd2-2450cb89ce77-156005074...,3266,2040.0,15,19,1,15891,44442,4493,1099640,1,1,400862,178970,140049,40156,2415,77550,1,15274,290542,1001057,13549,1,351016
2,158721,430,37,2,296,1,2196.0,336,254553,613672,0.0,24,117628,1032,286,4964,1.560089e+12,2,79726,1,0,139,232,0.0,0,8662,f49a740e-66c3-4605-9b67-4d3079fe69cb-156008914...,642,1080.0,5,7,1,11978,66533,10237,1099640,1,17,539581,71252,276898,15487,25024,698700,1011615,334610,15487,1001057,22231,1,11241
3,451641,498,32,1,152,1,0.0,336,16914,621281,0.0,19,308687,2115,287,3738,1.560063e+12,6,79726,0,1,96,2214,0.0,0,8897,fd60d096-f168-4540-b782-729d64d0fcc6-156006253...,2032,0.0,1,9,1,8567,59696,2406,1099640,1,2,400862,457846,135343,19344,95,258570,1011615,20709,290542,1001057,4212,1,11763
4,768430,127,70,2,195,1,2244.0,336,143681,579095,1.0,4,86480,341,95,2253,1.560079e+12,2,79726,0,0,140,1919,0.0,0,8675,a037b032-a5c7-40ea-9161-26b118b12406-156007938...,510,1080.0,13,7,2,2156,101284,17156,1099640,2,1,17763,2,1522,2337,1387,698700,1011615,158512,2337,1001057,2014,1,2337
5,731103,778,50,2,207,1,760.0,336,346191,651211,1.0,24,308687,2307,255,6590,1.560068e+12,2,79726,1,0,139,235,0.0,0,8669,7ac81a32-eefe-4fe9-9429-0ecb7be5d6b9-156006758...,1830,360.0,29,18,277927,51100,223009,4047,1099640,1,1,539581,457846,226658,207199,1605,698700,1011615,334610,199546,1001057,22394,1,70640
6,501327,234,85,2,96,0,1280.0,336,669419,276760,0.0,19,308687,1032,118,4479,1.560049e+12,2,79726,0,1,83,1638,0.0,0,5562,18deaade-64b3-4cc1-9fc8-e847f744d973-156004943...,3266,720.0,31,15,1,18705,22810,2450,1099640,1,1,400862,457846,276898,22650,11908,698700,1011615,69728,22650,1001057,7210,1,351016
7,731103,204,50,2,278,1,747.0,336,53979,678077,1.0,24,308687,492,255,790,1.560031e+12,2,79726,1,0,142,235,0.0,0,8666,a2d14df0-d499-4856-a330-8595f2923e21-156003109...,1383,360.0,2,10,277927,32026,223009,5712,1099640,1,1,539581,457846,140049,207199,907,698700,1011615,54227,199546,1001057,22215,1,307
8,731103,646,70,2,262,0,0.0,336,960906,627015,0.0,19,308687,2307,6,6589,1.560067e+12,6,79726,0,1,139,1461,0.0,0,3747,c0f81859-8256-4474-bb32-2f3ed3700e19-156006715...,3266,0.0,24,19,277927,22005,101284,11510,1099640,1,19,400862,457846,226658,78671,14682,258570,1011615,334610,4511,1001057,62573,1,351016
9,519573,89,5,2,156,1,1520.0,336,817007,284247,1.0,24,83934,1032,160,4954,1.560058e+12,3,79726,1,1,139,442,320.0,0,4008,db524c74-d9f7-41ed-bf2e-f38cdea4e15b-156005771...,3059,720.0,6,14,1,7484,7499,973,1099640,1,1,539581,1,276898,7484,10124,10277,1011615,334610,7946,1001057,1387,1,5066


In [0]:
print(train.loc[train['label'].isnull()].shape)
print( train.loc[train['label'].notnull()].shape)

(100000, 51)
(1000000, 51)


In [0]:
test = train.loc[train['label'].isnull()]

train = train.loc[train['label'].notnull()]
# train=train.append(test,ignore_index=True)

In [0]:
def mean_encode(train_data, test_data, columns, target_col, reg_method=None,
                alpha=0, add_random=False, rmean=0, rstd=0.1, folds=1):
    '''Returns a DataFrame with encoded columns'''
    encoded_cols = []
    target_mean_global = train_data[target_col].mean()
    for col in columns:
        # Getting means for test data
        nrows_cat = train_data.groupby(col)[target_col].count()
        target_means_cats = train_data.groupby(col)[target_col].mean()
        target_means_cats_adj = (target_means_cats*nrows_cat + 
                                 target_mean_global*alpha)/(nrows_cat+alpha)
        # Mapping means to test data
        encoded_col_test = test_data[col].map(target_means_cats_adj)
        # Getting a train encodings
        if reg_method == 'expanding_mean':
            train_data_shuffled = train_data.sample(frac=1, random_state=1)
            cumsum = train_data_shuffled.groupby(col)[target_col].cumsum() - train_data_shuffled[target_col]
            cumcnt = train_data_shuffled.groupby(col).cumcount()
            encoded_col_train = cumsum/(cumcnt)
            encoded_col_train.fillna(target_mean_global, inplace=True)
            if add_random:
                encoded_col_train = encoded_col_train + normal(loc=rmean, scale=rstd, 
                                                               size=(encoded_col_train.shape[0]))
        elif (reg_method == 'k_fold') and (folds > 1):
            kfold = StratifiedKFold(train_data[target_col].values, folds, shuffle=True, random_state=1)
            parts = []
            for tr_in, val_ind in kfold:
                # divide data
                df_for_estimation, df_estimated = train_data.iloc[tr_in], train_data.iloc[val_ind]
                # getting means on data for estimation (all folds except estimated)
                nrows_cat = df_for_estimation.groupby(col)[target_col].count()
                target_means_cats = df_for_estimation.groupby(col)[target_col].mean()
                target_means_cats_adj = (target_means_cats*nrows_cat + 
                                         target_mean_global*alpha)/(nrows_cat+alpha)
                # Mapping means to estimated fold
                encoded_col_train_part = df_estimated[col].map(target_means_cats_adj)
                if add_random:
                    encoded_col_train_part = encoded_col_train_part + normal(loc=rmean, scale=rstd, 
                                                                             size=(encoded_col_train_part.shape[0]))
                # Saving estimated encodings for a fold
                parts.append(encoded_col_train_part)
            encoded_col_train = pd.concat(parts, axis=0)
            encoded_col_train.fillna(target_mean_global, inplace=True)
        else:
            encoded_col_train = train_data[col].map(target_means_cats_adj)
            if add_random:
                encoded_col_train = encoded_col_train + normal(loc=rmean, scale=rstd, 
                                                               size=(encoded_col_train.shape[0]))

        # Saving the column with means
        encoded_col = pd.concat([encoded_col_train, encoded_col_test], axis=0)
        encoded_col[encoded_col.isnull()] = target_mean_global
        encoded_cols.append(pd.DataFrame({'mean_'+target_col+'_'+col:encoded_col}))
    all_encoded = pd.concat(encoded_cols, axis=1)
#     return all_encoded
    return (all_encoded.loc[train_data.index,:], 
            all_encoded.loc[test_data.index,:])

In [0]:
# train_temp=[[1,2,1],[3,4,0],[1,2,0],[3,4,1],[1,2,],[3,4,]]
# data=pd.DataFrame(train_temp,columns=['x1','x2','label'])
# train_temp=data[data['label'].notnull()]
# test_temp=data[data['label'].isnull()]
# tr,te=mean_encode(train_temp, test_temp, columns=['x1','x2'], target_col='label', reg_method='expanding_mean',
#                 alpha=0, add_random=False, rmean=0, rstd=0.1, folds=1)
# print(data)
# print(train_temp)
# print(test_temp)
# print(tr)
# print(te)

   x1  x2  label
0   1   2    1.0
1   3   4    0.0
2   1   2    0.0
3   3   4    1.0
4   1   2    NaN
5   3   4    NaN
   x1  x2  label
0   1   2    1.0
1   3   4    0.0
2   1   2    0.0
3   3   4    1.0
   x1  x2  label
4   1   2    NaN
5   3   4    NaN
   mean_label_x1  mean_label_x2
0            0.0            0.0
1            1.0            1.0
2            0.5            0.5
3            0.5            0.5
   mean_label_x1  mean_label_x2
4            0.5            0.5
5            0.5            0.5


In [0]:
train_mean,test_mean=mean_encode(train, test, columns=large_base_var, target_col='label', reg_method='expanding_mean',
                alpha=0, add_random=False, rmean=0, rstd=0.1, folds=1)

In [0]:
print(train_mean.shape)
print(train_mean.head())
print(test_mean.shape)
print(test_mean.head())

(1000000, 20)
   mean_label_adidmd5  mean_label_adunitshowid  ...  mean_label_sid  mean_label_ver
0            0.482894                 0.423611  ...        0.482894        0.672727
1            0.482894                 0.644693  ...        0.482894        0.257177
2            0.482894                 0.129536  ...        0.482894        0.124626
3            0.482894                 0.065217  ...        0.482894        0.060395
4            0.482894                 0.983162  ...        0.482894        0.915385

[5 rows x 20 columns]
(100000, 20)
         mean_label_adidmd5  ...  mean_label_ver
1000000            0.482894  ...        0.540969
1000001            0.482894  ...        0.367883
1000002            0.482894  ...        0.256221
1000003            0.482894  ...        0.540969
1000004            0.598411  ...        0.879082

[5 rows x 20 columns]


In [0]:
train_temp=pd.merge(train,train_mean,right_index=True,left_index=True)
test_temp=pd.merge(test,test_mean,right_index=True,left_index=True)


In [0]:
print(train_mean.shape)
print(train.shape)
print(train_mean.index[1099900:-1])

print(test_mean.shape)
print(test.shape)
print(test_mean.index[1099900:-1])

(1000000, 20)
(1000000, 51)
Int64Index([], dtype='int64')
(100000, 20)
(100000, 51)
Int64Index([], dtype='int64')


In [0]:
train_temp.head()

,h,ppi,w,day,hour,adidmd5_count,adunitshowid_count,apptype_count,city_count,idfamd5_count,imeimd5_count,ip_count,lan_count,macmd5_count,make_count,mediashowid_count,model_count,ntt_count,openudidmd5_count,osv_count,pkgname_count,province_count,reqrealip_count,sid_count,ver_count,mean_label_adidmd5,mean_label_adunitshowid,mean_label_apptype,mean_label_city,mean_label_idfamd5,mean_label_imeimd5,mean_label_ip,mean_label_lan,mean_label_macmd5,mean_label_make,mean_label_mediashowid,mean_label_model,mean_label_ntt,mean_label_openudidmd5,mean_label_osv,mean_label_pkgname,mean_label_province,mean_label_reqrealip,mean_label_sid,mean_label_ver
0,2340.0,3.0,1080.0,23,8,1,3814,15888,12370,1099640,1,1,539581,1,226658,15888,9492,698700,1011615,334610,15888,1001057,22584,1,11286,0.482894,0.423611,0.482549,0.468284,0.486202,0.482894,0.482894,0.639360,0.482894,0.430975,0.482549,0.498834,0.489174,0.496820,0.552925,0.482549,0.482601,0.369023,0.482894,0.672727
1,1080.0,0.0,2040.0,15,19,1,15891,44442,4493,1099640,1,1,400862,178970,140049,40156,2415,77550,1,15274,290542,1001057,13549,1,351016,0.482894,0.644693,0.648214,0.448594,0.482891,0.482894,0.482894,0.228221,0.675997,0.957455,0.652105,0.803406,0.830257,0.482894,0.931635,0.544265,0.479299,0.641912,0.482894,0.257177
2,2196.0,0.0,1080.0,5,7,1,11978,66533,10237,1099640,1,17,539581,71252,276898,15487,25024,698700,1011615,334610,15487,1001057,22231,1,11241,0.482894,0.129536,0.285482,0.560110,0.482239,0.482894,0.666667,0.635979,0.684947,0.347288,0.125603,0.387986,0.483054,0.493214,0.548999,0.125603,0.478766,0.767802,0.482894,0.124626
3,0.0,0.0,0.0,1,9,1,8567,59696,2406,1099640,1,2,400862,457846,135343,19344,95,258570,1011615,20709,290542,1001057,4212,1,11763,0.482894,0.065217,0.103198,0.369942,0.485469,0.482894,0.482894,0.230667,0.410945,0.189619,0.061813,0.333333,0.441927,0.496326,0.433130,0.544586,0.482001,0.112903,0.482894,0.060395
4,2244.0,0.0,1080.0,13,7,2,2156,101284,17156,1099640,2,1,17763,2,1522,2337,1387,698700,1011615,158512,2337,1001057,2014,1,2337,0.482894,0.983162,0.184850,0.482938,0.482995,0.482894,0.482894,0.922613,0.482894,0.997207,0.915385,0.996942,0.483680,0.493995,0.808250,0.915385,0.479500,0.934100,0.482894,0.915385


In [0]:
test_temp.sample(10)

,adidmd5,adunitshowid,apptype,carrier,city,dvctype,h,idfamd5,imeimd5,ip,label,lan,macmd5,make,mediashowid,model,nginxtime,ntt,openudidmd5,orientation,os,osv,pkgname,ppi,province,reqrealip,sid,ver,w,day,hour,adidmd5_count,adunitshowid_count,apptype_count,city_count,idfamd5_count,imeimd5_count,ip_count,lan_count,macmd5_count,make_count,mediashowid_count,model_count,ntt_count,openudidmd5_count,osv_count,pkgname_count,province_count,reqrealip_count,sid_count,ver_count,mean_label_adidmd5,mean_label_adunitshowid,mean_label_apptype,mean_label_city,mean_label_idfamd5,mean_label_imeimd5,mean_label_ip,mean_label_lan,mean_label_macmd5,mean_label_make,mean_label_mediashowid,mean_label_model,mean_label_ntt,mean_label_openudidmd5,mean_label_osv,mean_label_pkgname,mean_label_province,mean_label_reqrealip,mean_label_sid,mean_label_ver
1015071,317231,379,83,2,0,1,0.0,336,590834,808082,NaN,5,110533,2115,124,6397,1.560138e+12,6,79726,0,1,139,328,0.0,0,4678,714a2ef3-f136-4e21-bdaf-fe3add73e58a-156013795...,3148,0.0,21,6,1,7562,50348,16660,1099640,1,6,44812,1,135343,48508,9492,258570,1011615,334610,48620,1001057,54127,1,16496,0.482894,0.118364,0.469078,0.583492,0.482815,0.482894,0.400000,0.449170,0.482894,0.188325,0.483912,0.506739,0.443517,0.493846,0.547794,0.482619,0.479183,0.435592,0.482894,0.481022
1014094,559802,460,41,2,262,1,2280.0,336,679554,290445,NaN,24,210184,476,243,3548,1.560170e+12,2,79726,0,0,140,409,3.0,0,5588,1f9c9b6e-ca09-41d6-8356-518149a61ea0-156017025...,182,1080.0,30,2,1,3814,15888,11510,1099640,1,1,539581,1,50046,15888,1344,698700,1011615,158512,15888,1001057,22584,1,11286,0.482894,0.468634,0.492324,0.430759,0.482815,0.482894,0.482894,0.635883,0.482894,0.980495,0.492324,0.915410,0.483509,0.493846,0.806460,0.492324,0.479183,0.377035,0.482894,0.673455
1096509,449382,306,51,3,297,1,2240.0,336,235904,322093,NaN,22,292364,492,170,798,1.560161e+12,6,79726,0,1,140,1506,480.0,0,3837,2defdd2d-53b8-475e-9cce-2cc5a3786965-156016058...,2798,1080.0,10,3,1,4514,36714,15510,1099640,1,6,67489,32173,140049,32176,1612,258570,1011615,158512,32176,1001057,32176,1,22007,0.482894,0.925153,0.821143,0.447678,0.482815,0.482894,0.200000,0.660215,0.930776,0.957645,0.930781,0.922652,0.443517,0.493846,0.806460,0.930781,0.479183,0.930781,0.482894,0.929023
1052952,43307,718,13,3,270,0,1920.0,336,248117,383313,NaN,19,308687,1032,295,4556,1.560108e+12,2,79726,0,1,83,101,0.0,0,5545,0f2752d8-277d-47e1-9118-1d872699519a-156010824...,3266,1080.0,12,10,1,40171,87141,2961,1099640,1,1,400862,457846,276898,81975,12509,698700,1011615,69728,81975,1001057,4347,1,351016,0.482894,0.024159,0.064713,0.511628,0.482815,0.482894,0.482894,0.227360,0.406938,0.346866,0.065039,0.118520,0.483509,0.493846,0.111655,0.065039,0.479183,0.065703,0.482894,0.256221
1046970,2879,534,44,2,289,1,1920.0,336,693097,608146,NaN,24,60088,1032,51,4530,1.560163e+12,2,79726,0,0,101,2359,3.0,0,5588,d9e2d2bc-1034-41e5-a97d-ecca1d1ac209-156016304...,419,1080.0,7,15,1,2264,6248,1287,1099640,1,1,539581,1,276898,6248,8857,698700,1011615,58818,6248,1001057,22584,1,5375,0.482894,0.067994,0.071304,0.595745,0.482815,0.482894,0.482894,0.635883,0.482894,0.346866,0.071304,0.288830,0.483509,0.493846,0.139595,0.071304,0.479183,0.377035,0.482894,0.074338
1002422,662087,726,26,2,7,1,1920.0,336,258250,198356,NaN,24,114821,1032,43,4942,1.560114e+12,2,79726,0,0,139,2138,0.0,0,6235,52dddac5-8e82-4991-ac1f-d72ddf5d0f08-156011409...,3266,1080.0,19,2,1,13030,14217,12163,1099640,1,1,539581,1,276898,13030,15854,698700,1011615,334610,13030,1001057,2470,1,351016,0.482894,0.718527,0.727969,0.477497,0.482815,0.482894,0.482894,0.635883,0.482894,0.346866,0.718527,0.453944,0.483509,0.493846,0.547794,0.718527,0.479183,0.730821,0.482894,0.256221
1085199,731103,778,50,2,220,1,760.0,336,107291,324111,NaN,24,308687,476,255,823,1.560138e+12,2,79726,1,0,142,235,0.0,0,8665,7170f805-5659-4418-a934-d29793fc9eef-156013811...,1830,360.0,23,2,277927,51100,223009,739,1099640,1,1,539581,457846,50046,207199,3743,698700,1011615,54227,199546,100105

In [0]:

# test = train.loc[train['label'].isnull()]

# train = train.loc[train['label'].notnull()]

In [0]:
print(train.shape,test.shape)

NameError: ignored

In [0]:
test.sample(10)

ValueError: ignored

In [0]:
# test = train.loc[train['label'].isnull()]
# 
# train = train.loc[train['label'].notnull()]

In [0]:
# train=train_temp
# test=test_temp
# 删除没用的特征
drop = ['sid','label','nginxtime']+one_hot_var

y_train = train.loc[:,'label']
res = test.loc[:, ['sid']]

train.drop(drop, axis=1, inplace=True)
print('train:',train.shape)
test.drop(drop, axis=1, inplace=True)
print('test:',test.shape)

X_loc_train = train.values
y_loc_train = y_train.values
X_loc_test = test.values





train: (1000000, 22)
test: (100000, 22)


In [0]:
print(X_loc_train.shape)
print(X_loc_test.shape)

(1000000, 22)
(100000, 22)


In [0]:
print(feat_one_hot_sparse_matrix[:1000000].toarray().shape)
print(feat_one_hot_sparse_matrix[1000000:].shape)


(1000000, 14)
(100000, 14)


(100000, 14)

In [0]:
# feat_one_hot_sparse_matrix=enc.fit_transform(train[one_hot_var])

# from scipy import sparse
import numpy as np
X_loc_train=np.hstack((feat_one_hot_sparse_matrix[:1000000].toarray(),X_loc_train ))
X_loc_test=np.hstack((feat_one_hot_sparse_matrix[1000000:].toarray(), X_loc_test))

    
# for feature in cate_feature:
#     enc.fit(data[feature].values.reshape(-1, 1))
#     base_train_csr = sparse.hstack((base_train_csr, enc.transform(train_x[feature].values.reshape(-1, 1))), 'csr',
#                                    'bool')
#     base_predict_csr = sparse.hstack((base_predict_csr, enc.transform(predict[feature].values.reshape(-1, 1))),
#                                      'csr',
#                                      'bool')

In [0]:
train.sample(10)

,carrier,dvctype,h,orientation,os,ppi,w,day,hour,adidmd5_count,adunitshowid_count,apptype_count,city_count,idfamd5_count,imeimd5_count,ip_count,lan_count,macmd5_count,make_count,mediashowid_count,model_count,ntt_count,openudidmd5_count,osv_count,pkgname_count,province_count,reqrealip_count,sid_count,ver_count,mean_label_adidmd5,mean_label_adunitshowid,mean_label_apptype,mean_label_city,mean_label_idfamd5,mean_label_imeimd5,mean_label_ip,mean_label_lan,mean_label_macmd5,mean_label_make,mean_label_mediashowid,mean_label_model,mean_label_ntt,mean_label_openudidmd5,mean_label_osv,mean_label_pkgname,mean_label_province,mean_label_reqrealip,mean_label_sid,mean_label_ver
225162,1,1,2029.0,0,1,440.0,1080.0,20,8,2,3117,66533,3341,1099640,2,2,539581,2,53002,3260,14162,698700,1011615,158512,290542,1001057,2,1,2425,0.482894,0.034142,0.284148,0.515105,0.482533,0.482894,0.482894,0.636181,0.482894,0.810872,0.036082,0.921584,0.483419,0.493573,0.807768,0.542737,0.479015,0.482894,0.482894,0.036949
975385,2,0,2316.0,0,1,0.0,1080.0,13,16,1,40171,87141,3040,1099640,1,1,400862,457846,135343,81975,2857,698700,1011615,334610,81975,1001057,4499,1,351016,0.482894,0.024603,0.066606,0.501592,0.483243,0.482894,0.482894,0.229465,0.407859,0.187635,0.067800,0.566553,0.484846,0.494141,0.549343,0.067800,0.479862,0.066210,0.482894,0.258336
863797,2,1,760.0,1,0,0.0,360.0,5,4,277927,51100,223009,12370,1099640,1,1,539581,457846,276898,207199,15854,698700,1011615,334610,199546,1001057,22364,1,82768,0.599261,0.873648,0.797031,0.479695,0.483177,0.482894,0.482894,0.636223,0.407526,0.347519,0.817349,0.453437,0.483961,0.494239,0.548996,0.842490,0.479599,0.767406,0.482894,0.879210
894524,2,1,760.0,1,0,0.0,360.0,2,23,277927,54007,223009,4707,1099640,1,1,539581,457846,276898,207199,4293,698700,1011615,334610,199546,1001057,22364,1,82768,0.597202,0.870942,0.796307,0.460758,0.482291,0.482894,0.482894,0.635951,0.407018,0.347332,0.816463,0.460317,0.483163,0.493266,0.548967,0.841504,0.478848,0.770431,0.482894,0.877262
207560,2,2,1280.0,0,1,0.0,720.0,29,5,1,5747,66533,3174,1099640,1,4,26764,1,276898,5775,11908,258570,1011615,69728,290542,1001057,1634,1,19608,0.482894,0.278226,0.292009,0.549180,0.485416,0.482894,0.482894,0.557692,0.482894,0.355556,0.278226,0.105263,0.443683,0.496066,0.107266,0.538784,0.481954,0.430769,0.482894,0.517661
349170,2,1,760.0,1,0,0.0,360.0,6,12,277927,54007,223009,1218,1099640,1,1,539581,457846,276898,207199,4293,698700,1011615,334610,199546,1001057,22367,1,70640,0.598425,0.870882,0.796822,0.524046,0.483008,0.482894,0.482894,0.636363,0.407298,0.348030,0.817037,0.464822,0.483634,0.494021,0.549428,0.842126,0.479516,0.767003,0.482894,0.885910
808795,2,1,1344.0,0,1,0.0,720.0,24,1,1,2932,11362,1739,1099640,1,1,67489,457846,16503,8967,177,698700,1011615,42204,8967,1001057,436,1,351016,0.482894,0.788805,0.429728,0.504072,0.483016,0.482894,0.482894,0.660084,0.407348,0.861802,0.519428,0.983333,0.483819,0.494027,0.330587,0.519428,0.479424,0.409677,0.482894,0.256786
19897,2,1,2222.0,0,0,3.0,1080.0,27,16,1,2441,3313,6908,1099640,1,6,539581,457846,50046,2441,4007,258570,1011615,334610,1982,1001057,2265,1,1789,0.482894,0.845283,0.824092,0.438780,0.482372,0.482894,0.666667,0.636150,0.406609,0.982535,0.845283,0.928226,0.441897,0.493184,0.549034,0.812403,0.478894,0.375346,0.482894,0.865052
99304,1,1,0.0,0,1,0.0,0.0,11,1,1,8567,59696,2838,1099640,1,1,400862,457846,135343,19344,3685,698700,1011615,42204,290542,1001057,4411,1,11763,0.482894,0.072585,0.104901,0.477237,0.482437,0.482894,0.482894,0.228508,0.406920,0.186967,0.069933,0.254109,0.483858,0.493220,0.333439,0.543311,0.478987,0.134951,0.482894,0.066251
577936,2,1,760.0,1,0,0.0,360.0,3,11,277927,53437,223009,1759,1099640,2,1,539581,71252,226658,207199,14682,698700,1011615,334610,199546,1001057,22364,1,82768,0.598132,0.872203,0.796711,0.469880,0.482880,0.482894,0.482894,0.636343,0.686893,0.430258,0.816801,0.675037,0.483520,0.493906,0.549440,0.841968,0.479433,0.767835,0.482894,0.877959


In [0]:
# res=test.loc[:, ['sid']]
# X_loc_train = train.values[0:100]
# y_loc_train = y_train.values[0:100]
# X_loc_test = test.values[0:100]

In [0]:
# res=res
# X_loc_train = train.values
# y_loc_train = y_train.values
# X_loc_test = test.values

In [0]:
# import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
import numpy as np 
import pandas as pd
import time
import datetime
import gc
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, log_loss


'''
from sklearn.model_selection import KFold
   ...: import numpy as np
   ...: X = np.arange(24).reshape(12,2)
   ...: y = np.random.choice([1,2],12,p=[0.4,0.6])
   ...: kf = KFold(n_splits=5,shuffle=False)
   ...: for train_index , test_index in kf.split(X):
   
from sklearn.model_selection import StratifiedKFold
import numpy as np

X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1])
skf = StratifiedKFold(n_splits=2)
skf.get_n_splits(X, y)

> 2

for train_index, test_index in skf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

> TRAIN: [1 3] TEST: [0 2]
TRAIN: [0 2] TEST: [1 3]

'''

# 模型部分
model = LGBMClassifier(boosting_type='gbdt', num_leaves=48, max_depth=-1, learning_rate=0.05, n_estimators=3000,
                           max_bin=425, subsample_for_bin=50000, objective='binary', min_split_gain=0,
                           min_child_weight=5, min_child_samples=10, subsample=0.8, subsample_freq=1,
                           colsample_bytree=1, reg_alpha=3, reg_lambda=5, seed=1000, n_jobs=10, silent=False,verbose=0)

# 五折交叉训练，构造五个模型
skf=StratifiedKFold( n_splits=5, shuffle=True, random_state=1024)
skf.get_n_splits(X_loc_train, y_loc_train)
baseloss = []
loss = 0
for i, (train_index, test_index) in enumerate(skf.split(X_loc_train, y_loc_train)):
    print("Fold", i)
    lgb_model = model.fit(X_loc_train[train_index], y_loc_train[train_index],
                          eval_names =['train','valid'],
                          eval_metric='logloss',
                          eval_set=[(X_loc_train[train_index], y_loc_train[train_index]), 
                                    (X_loc_train[test_index], y_loc_train[test_index])],early_stopping_rounds=100)
    baseloss.append(lgb_model.best_score_['valid']['binary_logloss'])
    loss += lgb_model.best_score_['valid']['binary_logloss']
    test_pred= lgb_model.predict_proba(X_loc_test, num_iteration=lgb_model.best_iteration_)[:, 1]
    print('test mean:', test_pred.mean())
    res['prob_%s' % str(i)] = test_pred
print('logloss:', baseloss, loss/5)

# 加权平均
res['label'] = 0
for i in range(5):
    res['label'] += res['prob_%s' % str(i)]
res['label'] = res['label']/5
res[['sid', 'label']].to_csv("lgb_baseline_%s_prob.csv" % now, index=False)
res['label']=res['label'].map(lambda row:1 if row>=0.5  else 0)

# 提交结果
mean = res['label'].mean()
print('mean:',mean)
now = datetime.datetime.now()
now = now.strftime('%m-%d-%H-%M')
res[['sid', 'label']].to_csv("lgb_baseline_%s.csv" % now, index=False)
print(res[['sid', 'label']].head())

[162]	train's binary_logloss: 0.188858	train's binary_logloss: 0.188858	valid's binary_logloss: 0.190887	valid's binary_logloss: 0.190887
[163]	train's binary_logloss: 0.188711	train's binary_logloss: 0.188711	valid's binary_logloss: 0.190742	valid's binary_logloss: 0.190742
[164]	train's binary_logloss: 0.188511	train's binary_logloss: 0.188511	valid's binary_logloss: 0.190552	valid's binary_logloss: 0.190552
[165]	train's binary_logloss: 0.188293	train's binary_logloss: 0.188293	valid's binary_logloss: 0.190327	valid's binary_logloss: 0.190327
[166]	train's binary_logloss: 0.18811	train's binary_logloss: 0.18811	valid's binary_logloss: 0.190153	valid's binary_logloss: 0.190153
[167]	train's binary_logloss: 0.187834	train's binary_logloss: 0.187834	valid's binary_logloss: 0.189871	valid's binary_logloss: 0.189871
[168]	train's binary_logloss: 0.187664	train's binary_logloss: 0.187664	valid's binary_logloss: 0.189711	valid's binary_logloss: 0.189711
[169]	train's binary_logloss: 0.1875

In [0]:
pip list

Package                  Version              
------------------------ ---------------------
absl-py                  0.7.1                
alabaster                0.7.12               
albumentations           0.1.12               
altair                   3.1.0                
astor                    0.8.0                
astropy                  3.0.5                
atari-py                 0.1.15               
atomicwrites             1.3.0                
attrs                    19.1.0               
audioread                2.1.8                
autograd                 1.2                  
Babel                    2.7.0                
backcall                 0.1.0                
backports.tempfile       1.0                  
backports.weakref        1.0.post1            
beautifulsoup4           4.6.3                
bleach                   3.1.0                
blis                     0.2.4                
bokeh                    1.0.4                
boto         

In [0]:
# lan	400862	36.44%
# 27	ver	351016	31.91%
# 13	make	135343	12.30%
# 4	city	16660	1.51%
# 21	osv	13929	1.27%
# 15	model	4562	0.41%
for each in ['lan','ver','make','city','osv','model']:
  print(train[each].mode())
  train[each]=train[each].fillna(train[each].mode()) 

0    zh-CN
dtype: object
0    30927000
dtype: object
0    OPPO
dtype: object
0    北京市
dtype: object
0    8.1.0
dtype: object
0    PBAM00
dtype: object


In [0]:
df_null_ratio=null_ratio(train)

字段名为： adidmd5    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： adunitshowid 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： apptype    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： carrier    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： city       缺失值数量: 16660 缺失数量占比： 1.51%
字段名为： dvctype    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： h          缺失值数量: 0    缺失数量占比： 0.00%
字段名为： idfamd5    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： imeimd5    缺失值数量: 0    缺失数量占比： 0.00%
字段名为： ip         缺失值数量: 0    缺失数量占比： 0.00%
字段名为： label      缺失值数量: 100000 缺失数量占比： 9.09%
字段名为： lan        缺失值数量: 400862 缺失数量占比： 36.44%
字段名为： macmd5     缺失值数量: 0    缺失数量占比： 0.00%
字段名为： make       缺失值数量: 135343 缺失数量占比： 12.30%
字段名为： mediashowid 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： model      缺失值数量: 4562 缺失数量占比： 0.41%
字段名为： nginxtime  缺失值数量: 0    缺失数量占比： 0.00%
字段名为： ntt        缺失值数量: 0    缺失数量占比： 0.00%
字段名为： openudidmd5 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： orientation 缺失值数量: 0    缺失数量占比： 0.00%
字段名为： os         缺失值数量: 0    缺失数量占比： 0.00%
字段名为： osv        缺失值数量: 13929 缺失数量占比： 1.27%
字段名为： pkgname    缺失值数量: 0    缺失数量占比： 0.

In [0]:
df_null_ratio

,字段名为,缺失值数量,缺失数量占比
10,label,100000,9.09%
11,lan,400862,36.44%
27,ver,351016,31.91%
13,make,135343,12.30%
4,city,16660,1.51%
21,osv,13929,1.27%
15,model,4562,0.41%
0,adidmd5,0,0.00%
19,orientation,0,0.00%
20,os,0,0.00%


In [0]:
train[['lan','ver','make','city','osv','model']].sample(10)


,lan,ver,make,city,osv,model
483651,zh-CN,5.76.2.6060,HUAWEI,苏州市,8.1.0,JKM-TL00
27763,NaN,190605,OPPO,郑州市,8.1.0,PADM00
582659,zh-CN,30924000,HONOR,玉林市,8.1.0,COL AL10
352729,zh-CN,8.3.5.2,OPPO,郑州市,8.1.0,PBEM00
783286,NaN,4.9.10,NaN,沈阳市,8.1,PACM00
483620,zh-CN,5.76.2.6060,HUAWEI,怀化市,9,LYA-AL00
150486,cn,8.3.70,NaN,曲靖市,7.1.2,M850
86194,zh-CN,521000,vivo,丽江市,8.1.0,V1813BT
310932,zh-CN,5.4.0,Meizu,长沙市,5.1,M1 METAL
677509,zh-CN,30927000,Xiaomi,青岛市,9.0.0,MI 8 LITE


In [0]:

import pandas as pd
import numpy as np
#显示所有列
pd.set_option("display.max_columns",500)

In [0]:
path = "../data/"
# print(os.listdir(path))
train = pd.read_csv(path+"train.csv",encoding='utf8'
                   ) #100W
test = pd.read_csv(path+"test.csv",encoding='utf8',parse_dates=['auditing_date', 'due_date']) #13W
train=train.append(test)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [0]:
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1130000 entries, 0 to 129999
Data columns (total 7 columns):
auditing_date    1130000 non-null object
due_amt          1130000 non-null float64
due_date         1130000 non-null object
listing_id       1130000 non-null int64
repay_amt        1000000 non-null object
repay_date       1000000 non-null object
user_id          1130000 non-null int64
dtypes: float64(1), int64(2), object(4)
memory usage: 322.2 MB


In [0]:
train.sample(10)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id
464480,2018-04-23,612.9068,2018-05-23,3144810,612.9068,2018-05-23,241863
706244,2018-12-01,198.9431,2019-01-01,5021198,198.9431,2018-12-18,118962
586764,2018-06-01,572.4266,2018-07-01,3602911,\N,\N,411162
769786,2018-07-22,74.1767,2018-08-22,4051347,74.1767,2018-08-22,657669
22064,2018-05-21,1701.9015,2018-06-21,3482276,1701.9015,2018-06-21,400035
203966,2018-05-16,107.6503,2018-06-16,3414983,107.6503,2018-06-11,483836
23290,2019-02-02 00:00:00,258.7045,2019-03-02 00:00:00,5289416,NaN,NaN,622946
77836,2019-02-07 00:00:00,84.7085,2019-03-07 00:00:00,5307459,NaN,NaN,863651
789531,2018-12-18,377.0507,2019-01-18,5143208,377.0507,2019-01-17,530261
745352,2018-11-24,483.4584,2018-12-24,4977641,483.4584,2018-12-24,712673


In [0]:
def replace_na_date(x):
  if x=='\\N'or pd.isnull(x):
    return '2080-01-01'
  else:
    return x
def replace_na_amt(x):
  if x=='\\N'or pd.isnull(x):
    return 0
  else :
    return x
train['repay_date']=train['repay_date'].map(replace_na_date)
train['repay_amt']=train['repay_amt'].map(replace_na_amt)

In [0]:
#convert to uint32
for each in ['user_id',	'listing_id']:
  train[each]=train[each].astype(np.uint32)

In [0]:
#convert to datatime
train['auditing_date']=pd.to_datetime(train['auditing_date'],format='%Y-%m-%d')
train['due_date']=pd.to_datetime(train['due_date'],format='%Y-%m-%d')
train['repay_date']=pd.to_datetime(train['repay_date'],format='%Y-%m-%d')

In [0]:
#convert to float32
for each in['due_amt','repay_amt']:
  train[each]=train[each].astype(np.float32)
  

In [0]:
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1130000 entries, 0 to 129999
Data columns (total 7 columns):
auditing_date    1130000 non-null datetime64[ns]
due_amt          1130000 non-null float32
due_date         1130000 non-null datetime64[ns]
listing_id       1130000 non-null uint32
repay_amt        1130000 non-null float32
repay_date       1130000 non-null datetime64[ns]
user_id          1130000 non-null uint32
dtypes: datetime64[ns](3), float32(2), uint32(2)
memory usage: 51.7 MB


In [0]:
train.sample(10)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id
971276,2018-06-25,107.602699,2018-07-25,3835503,107.602699,2018-07-23,171803
495253,2018-03-26,1134.348633,2018-04-26,2774822,1134.348633,2018-04-26,67671
866086,2018-06-25,194.459702,2018-07-25,3832894,0.000000,2080-01-01,794746
868439,2018-01-25,255.276093,2018-02-25,2115683,255.276093,2018-02-24,58154
205951,2018-10-17,462.268188,2018-11-17,4671029,462.268188,2018-11-17,852558
422837,2018-11-02,458.325806,2018-12-02,4819629,458.325806,2018-12-02,562528
73177,2019-02-11,144.531296,2019-03-11,5322145,0.000000,2080-01-01,845146
89546,2019-02-27,898.467407,2019-03-27,5384674,0.000000,2080-01-01,476788
227372,2018-12-13,1600.046753,2019-01-13,5111466,1600.046753,2019-01-13,119936
267368,2018-01-31,97.300301,2018-02-28,2179315,97.300301,2018-02-25,28039


In [0]:
submission = pd.read_csv(path+"/submission/submission.csv",encoding='utf8',parse_dates=[ 'repay_date'],##字符串似乎占空间是最大的
                        dtype={'user_id':np.uint32, 'listing_id':np.uint32,'repay_amt':np.float32
                               }) #398W

In [0]:
submission.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130000 entries, 0 to 129999
Data columns (total 3 columns):
listing_id    130000 non-null uint32
repay_amt     117675 non-null float32
repay_date    117675 non-null datetime64[ns]
dtypes: datetime64[ns](1), float32(1), uint32(1)
memory usage: 2.0 MB


In [0]:
listing_info = pd.read_csv(path+"listing_info.csv",encoding='utf8',parse_dates=['auditing_date'],
                          dtype={'user_id':np.uint32, 'listing_id':np.uint32,'term':np.uint32,'rate':np.float32,'principal':np.float32 
                                #'due_amt':np.float32,'repay_amt':np.float32
                               })

In [0]:
listing_info.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5484891 entries, 0 to 5484890
Data columns (total 6 columns):
user_id          uint32
listing_id       uint32
auditing_date    datetime64[ns]
term             uint32
rate             float32
principal        float32
dtypes: datetime64[ns](1), float32(2), uint32(3)
memory usage: 146.5 MB


In [0]:
listing_info.sample(10)

,user_id,listing_id,auditing_date,term,rate,principal
3948863,594163,4781382,2018-10-29,3,7.2,1610.0
4383776,377263,4565882,2018-10-01,6,8.3,3350.0
4899155,467246,3834112,2018-06-25,3,7.2,630.0
2885252,294758,5232854,2019-01-12,6,6.9,1270.0
2400727,919952,4948312,2018-11-20,6,7.2,3900.0
3394357,464802,767027,2017-08-14,12,9.3,1460.0
765308,137003,3856186,2018-06-27,6,8.0,5540.0
4767476,617770,767202,2017-08-14,12,9.3,1170.0
3159948,658988,2584450,2018-03-10,6,8.0,850.0
3320143,832379,4120966,2018-08-01,9,7.2,630.0


In [0]:
user_info = pd.read_csv(path+"user_info.csv",encoding='utf8',parse_dates=['insertdate','reg_mon'],
                        dtype={'user_id':np.uint32, 'age':np.uint32,
                               })

In [0]:
user_info.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954209 entries, 0 to 954208
Data columns (total 8 columns):
user_id          954209 non-null uint32
reg_mon          954209 non-null datetime64[ns]
gender           954209 non-null object
age              954209 non-null uint32
cell_province    954209 non-null object
id_province      954209 non-null object
id_city          954209 non-null object
insertdate       954209 non-null datetime64[ns]
dtypes: datetime64[ns](2), object(4), uint32(2)
memory usage: 264.8 MB


In [0]:
user_info.head()

,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate
0,483833,2017-04-01,男,19,c29,c26,c26241,2018-12-11
1,156772,2016-05-01,男,31,c11,c11,c11159,2018-02-13
2,173388,2016-05-01,男,34,c02,c02,c02182,2018-08-21
3,199107,2016-07-01,女,25,c09,c09,c09046,2018-06-05
4,122560,2016-03-01,男,23,c05,c05,c05193,2018-04-02


In [0]:
#labelencoder+onehotencoder
from sklearn.preprocessing import LabelEncoder

In [0]:
user_info['gender']=LabelEncoder().fit_transform(user_info['gender'])

In [0]:
user_info['gender']=user_info['gender'].astype(np.uint32)

In [0]:
#fill null value and convert dtype
for each in ['cell_province',	'id_province',	'id_city']:
  user_info[each]=user_info[each].map(lambda x:np.nan if x=='\\N' else x )
  user_info[each].fillna(user_info[each].mode()[0], inplace=True)
  user_info[each]=user_info[each].map(lambda x:x[1:] ).astype(np.uint32)


In [0]:
user_info.head(10)#1代表男，0代表女

,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate
0,483833,2017-04-01,1,19,29,26,26241,2018-12-11
1,156772,2016-05-01,1,31,11,11,11159,2018-02-13
2,173388,2016-05-01,1,34,2,2,2182,2018-08-21
3,199107,2016-07-01,0,25,9,9,9046,2018-06-05
4,122560,2016-03-01,1,23,5,5,5193,2018-04-02
5,550399,2017-06-01,1,30,8,7,7282,2018-04-19
6,414650,2017-02-01,1,53,12,12,23141,2018-10-16
7,275042,2016-10-01,0,23,21,21,21174,2018-07-10
8,184180,2016-06-01,1,29,4,4,4348,2018-02-08
9,817503,2018-03-01,1,25,6,25,25199,2018-11-27


In [0]:
user_info.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954209 entries, 0 to 954208
Data columns (total 8 columns):
user_id          954209 non-null uint32
reg_mon          954209 non-null datetime64[ns]
gender           954209 non-null uint32
age              954209 non-null uint32
cell_province    954209 non-null uint32
id_province      954209 non-null uint32
id_city          954209 non-null uint32
insertdate       954209 non-null datetime64[ns]
dtypes: datetime64[ns](2), uint32(6)
memory usage: 36.4 MB


In [0]:
user_taglist = pd.read_csv(path+"user_taglist.csv",encoding='utf8',parse_dates=['insertdate'],dtype={'user_id':np.uint32})

In [0]:
def property_feature(org):
    tmp=org['taglist'].apply(lambda x:x.split('|')).values
    property_dict={}
    property_list=[]
    for i in tmp:
        property_list+=i
    for i in property_list:
        if i in property_dict:
            property_dict[i]+=1
        else:
            property_dict[i] = 1
#     print(property_dict)
#     print('dict finish')
    def top(x):
        propertys=x.split('|')
        cnt=[property_dict[i] for i in propertys]
        res=sorted(zip(propertys,cnt),key=lambda x:x[1],reverse=True)#字典按照次数进行排序
#         print('res',res)
        top1=res[0][0]
        top2 = '_'.join([i[0] for i in res[:2]])
        top3 = '_'.join([i[0] for i in res[:3]])
        top4 = '_'.join([i[0] for i in res[:4]])
        top5='_'.join([i[0] for i in res[:5]])
        top10 = '_'.join([i[0] for i in res[:10]])
        return (top1,top2,top3,top4,top5,top10)
    org['top']=org['taglist'].apply(top)###运用top函数
    print('top finish')
    org['top1']=org['top'].apply(lambda x:x[0])
    org['top2'] = org['top'].apply(lambda x: x[1])
    org['top3'] = org['top'].apply(lambda x: x[2])
    org['top4'] = org['top'].apply(lambda x: x[3])
    org['top5'] = org['top'].apply(lambda x: x[4])
    org['top10'] = org['top'].apply(lambda x: x[5])
#     print(org)
#     org.to_csv('org.csv',index=False)
    return org[['user_id','top1','top2','top3','top4','top5','top10']]

In [0]:
user_taglist_new_feature= property_feature(user_taglist)

top finish


In [0]:
user_taglist_new_feature.sample(10)

,user_id,top1,top2,top3,top4,top5,top10
151137,708207,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_70_1876_5628_631
480018,220953,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_631_2642_2300_2431_1234
247312,839093,2017,2017_2710,2017_2710_1804,2017_2710_1804_1654,2017_2710_1804_1654_1876,2017_2710_1804_1654_1876_5628_727_2580_719_4185
360597,479212,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
227893,20124,2017,2017_1804,2017_1804_1654,2017_1804_1654_440,2017_1804_1654_440_70,2017_1804_1654_440_70_631_1478_2164_2365_2500
125532,781757,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
153384,482249,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_5628_631
142624,381417,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_1876_5628_727_2300_2580
189233,503441,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
212714,616596,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_5628_631_2580


In [0]:
import gc
del user_taglist['taglist']
del user_taglist['top']
gc.collect()

KeyError: ignored

In [0]:
user_taglist.sample(10)

,user_id,insertdate,top1,top2,top3,top4,top5,top10
413424,31006,2018-12-03,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_440,2017_2710_1804_1911_440_70_1876_5628_727_2300
324805,522535,2018-11-17,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_70_1876_5628_631
281087,437244,2018-01-30,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_631
125556,780127,2018-08-12,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
18873,618028,2018-05-24,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
133211,146928,2018-10-21,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628
285571,831596,2018-12-15,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_440_631_2300_4185_1478
376647,837042,2018-07-05,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_340,2017_2710_1804_1911_340_440_70_1876_5628_631
266095,426205,2018-07-06,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_631_727
113889,826912,2018-12-22,2017,2017_2710,2017_2710_1804,2017_2710_1804_1911,2017_2710_1804_1911_1654,2017_2710_1804_1911_1654_340_440_70_1876_5628


In [0]:
#lablencoder
for each in ['top1','top2','top3','top4','top5','top10']:
  user_taglist[each]=LabelEncoder().fit_transform(user_taglist[each]).astype(np.uint32)

107

In [0]:
user_taglist.sample(10)

,user_id,insertdate,top1,top2,top3,top4,top5,top10
540182,459083,2018-11-19,18,146,538,1093,1606,2905
130422,794097,2018-12-03,18,166,687,1898,4308,38443
516696,221699,2018-10-01,18,166,687,1898,4308,38582
431996,51262,2018-04-25,18,166,687,1898,4308,38443
546230,791463,2018-06-21,18,166,687,1898,4308,39982
175985,293275,2018-05-27,18,166,687,1898,4308,38445
322076,837538,2018-11-07,18,166,687,1898,4308,37847
537118,115138,2018-12-11,18,166,687,1894,4238,22330
409934,284485,2018-12-21,18,166,687,1898,4308,37847
228941,389673,2018-11-26,18,149,590,1485,2916,11055


In [0]:
user_taglist.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615160 entries, 0 to 615159
Data columns (total 8 columns):
user_id       615160 non-null uint32
insertdate    615160 non-null datetime64[ns]
top1          615160 non-null uint32
top2          615160 non-null uint32
top3          615160 non-null uint32
top4          615160 non-null uint32
top5          615160 non-null uint32
top10         615160 non-null uint32
dtypes: datetime64[ns](1), uint32(7)
memory usage: 21.1 MB


In [0]:
user_taglist['top5'].nunique()

6268

In [0]:
user_behavior_logs = pd.read_csv(path+"user_behavior_logs.csv",encoding='utf8',parse_dates=['behavior_time'],
                                 dtype={'user_id':np.uint32,'behavior_type':np.uint32}) #1G

In [0]:
user_behavior_logs.info(memory_usage='deep')

NameError: ignored

In [0]:
user_repay_logs = pd.read_csv(path+"user_repay_logs.csv",encoding='utf8',
                              parse_dates=[ 'due_date', 'repay_date'],
                    dtype={'user_id':np.uint32, 'listing_id':np.uint32,'order_id':np.uint32,  
                                'due_amt':np.float32,'repay_amt':np.float32}) #900M

In [0]:
user_repay_logs.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18001297 entries, 0 to 18001296
Data columns (total 7 columns):
user_id       uint32
listing_id    uint32
order_id      uint32
due_date      datetime64[ns]
due_amt       float32
repay_date    datetime64[ns]
repay_amt     float32
dtypes: datetime64[ns](2), float32(2), uint32(3)
memory usage: 618.0 MB


In [0]:

# train = pd.read_csv(path+"train.csv",encoding='utf8',parse_dates=['auditing_date', 'due_date', 'repay_date'],
#                     dtype={'user_id':np.uint32, 'listing_id':np.uint32, 
#                                 #'due_amt':np.float32,'repay_amt':np.float32
#                                }
#                    ) #100W
# test = pd.read_csv(path+"test.csv",encoding='utf8',parse_dates=['auditing_date', 'due_date']) #13W
# submission = pd.read_csv(path+"/submission/submission.csv",encoding='utf8',
#                         dtype={'user_id':np.uint32, 'listing_id':np.uint32,'repay_date':str,'repay_amt':np.float32
#                                }) #398W
# listing_info = pd.read_csv(path+"listing_info.csv",encoding='utf8',
#                           dtype={'user_id':np.uint32, 'listing_id':np.uint32,'term':np.uint32,'rate':np.floate32,'principal':np.float32 
#                                 #'due_amt':np.float32,'repay_amt':np.float32
#                                })
# user_info = pd.read_csv(path+"user_info.csv",encoding='utf8',dtype={'user_id':np.uint32, 'age':np.uint32,
#                                })
# user_taglist = pd.read_csv(path+"user_taglist.csv",encoding='utf8',dtype={'user_id':np.uint32})
# user_behavior_logs = pd.read_csv(path+"user_behavior_logs.csv",encoding='utf8') #1G
# user_repay_logs = pd.read_csv(path+"user_repay_logs.csv",encoding='utf8') #900M


In [0]:

del user_repay_logs['user_id']
gc.collect()


398

In [0]:
listing_repay=pd.merge(listing_info,user_repay_logs,on=['listing_id'],how='left')

In [0]:

del listing_info
del user_repay_logs
gc.collect()

28

In [0]:
listing_repay.sample(10)

,user_id,listing_id,auditing_date,term,rate,principal,order_id,due_date,due_amt,repay_date,repay_amt
14097794,443831,3311890,2018-05-07,12,8.6,1170.0,2.0,2018-07-07,102.102203,2018-07-07,102.102203
3551408,267412,3574502,2018-05-30,9,7.2,4730.0,5.0,2018-10-30,541.447876,2018-10-26,541.447876
3348207,120594,1923029,2018-01-06,6,7.2,7010.0,1.0,2018-02-06,1192.990601,2018-02-06,1192.990601
6074815,152492,4385054,2018-09-07,12,8.6,4100.0,1.0,2018-10-07,357.794098,2018-10-07,357.794098
3658456,538156,3542567,2018-05-27,9,7.2,910.0,2.0,2018-07-27,104.168602,2018-07-27,104.168602
858850,110514,4537227,2018-09-26,3,7.2,1710.0,3.0,2018-12-26,576.853577,2018-12-26,576.853577
128334,178962,1440865,2017-11-13,6,8.0,1390.0,4.0,2018-03-13,237.104202,2018-03-13,237.104202
4459205,8041,3301151,2018-05-06,12,8.6,3660.0,7.0,2018-12-06,319.396698,2018-11-16,319.396698
12503738,453602,2073633,2018-01-22,12,7.6,1170.0,1.0,2018-02-22,101.560699,2018-02-06,101.560699
2632783,163874,619041,2017-07-22,12,9.3,2260.0,3.0,2017-10-22,197.954803,2017-09-12,197.954803


In [0]:
sy_listing_id=list(set(listing_repay['listing_id'])-set(train['listing_id']))

In [0]:
sy_listing_id[:10]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [0]:
sy_listing_repay=listing_repay[listing_repay['listing_id'].isin(sy_listing_id)]

In [0]:
sy_listing_repay.sample(5)

,user_id,listing_id,auditing_date,term,rate,principal,order_id,due_date,due_amt,repay_date,repay_amt
3991050,366514,211535,2017-04-17,12,9.3,3600.0,8.0,2017-12-17,315.326294,2017-12-12,315.326294
10771951,88709,1117133,2017-10-06,9,7.6,2260.0,1.0,2017-11-06,259.131104,2017-11-06,259.131104
4302101,161509,126049,2017-03-01,12,9.3,2500.0,8.0,2017-11-01,218.976593,2017-10-31,218.976593
1679664,751917,4221268,2018-08-16,6,8.3,1840.0,5.0,2019-01-16,314.135895,2018-10-03,314.135895
3094354,375110,1998393,2018-01-14,9,7.2,900.0,4.0,2018-05-14,103.023903,2200-01-01,103.023903


In [0]:
for each in  ['user_id','auditing_date','due_date',	'due_amt',	'repay_date',	'repay_amt']:
    del listing_repay[each]
gc.collect()

57

In [0]:
train=pd.merge(train,listing_repay,on=['listing_id'],how='left')

In [0]:
del listing_repay
gc.collect()

35

In [0]:
train.sample(10)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id,term,rate,principal,order_id
15450,2018-07-30,2210.903809,2018-08-30,4098952,2210.903809,2018-08-28,133894,6,8.3,12950.0,NaN
600202,2018-03-14,824.191406,2018-04-14,2625136,0.000000,2080-01-01,763062,9,7.2,7200.0,NaN
1470579,2018-02-06,151.463806,2018-03-06,2234222,151.463806,2018-03-05,151596,6,7.2,890.0,4.0
190856,2018-06-04,109.892097,2018-07-04,3635664,109.892097,2018-06-15,383232,9,7.2,960.0,NaN
35158,2018-01-19,761.232300,2018-02-19,2047861,761.232300,2018-02-19,331376,9,7.2,6650.0,6.0
1436834,2018-12-07,111.036797,2019-01-07,5066742,111.036797,2018-12-23,253190,9,7.2,970.0,NaN
47262,2018-10-11,394.689301,2018-11-11,4630947,394.689301,2018-10-12,910954,3,7.2,1170.0,NaN
9068,2018-04-01,258.704498,2018-05-01,2847931,258.704498,2018-05-01,533099,9,7.2,2260.0,NaN
753398,2018-10-02,544.616272,2018-11-02,4574631,544.616272,2018-10-08,239575,6,8.3,3190.0,NaN
1067851,2018-03-11,571.438721,2018-04-11,2591047,571.438721,2018-04-11,437155,6,8.0,3350.0,2.0


In [0]:
train=train.append(sy_listing_repay)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [0]:
train.sample(20)

NameError: ignored

In [0]:
train=pd.merge(train,user_info,on=['user_id'],how='left')

In [0]:
del user_info
gc.collect()

45

In [0]:
train['order_id']=train['order_id'].astype(np.float32)

In [0]:
train.sample(15)

,auditing_date,due_amt,due_date,listing_id,order_id,principal,rate,repay_amt,repay_date,term,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate
16000905,2018-04-22,578.761719,2018-08-22,3124854,4.0,3390.0,8.3,578.761719,2200-01-01,6,623092,2017-08-01,1,31,4,11,11077,2018-11-02
15350828,2017-12-13,205.725494,2018-04-13,1720097,4.0,2370.0,7.6,205.725494,2018-03-15,12,469341,2017-04-01,0,38,4,4,4206,2018-03-15
16659530,2018-10-08,155.360596,2018-12-08,4610306,2.0,910.0,8.3,155.360596,2018-12-08,6,819229,2018-03-01,0,31,12,4,4348,2019-03-03
3339821,2018-03-05,963.635620,2018-05-05,2520263,2.0,11060.0,8.3,963.635620,2018-05-05,12,150093,2016-04-01,1,32,8,8,8256,2018-06-05
10707829,2017-12-11,252.599701,2018-01-11,1699279,1.0,2910.0,7.6,252.599701,2018-01-05,12,714026,2017-11-01,1,27,22,12,12190,2018-06-14
11293297,2017-11-25,660.440613,2018-02-25,1555619,3.0,5760.0,7.6,660.440613,2018-02-21,9,142981,2016-04-01,1,31,25,25,25140,2018-05-27
10539521,2017-12-17,141.944000,2018-07-17,1744515,7.0,1240.0,7.2,141.944000,2018-07-17,9,309117,2016-11-01,1,29,8,31,31067,2018-07-23
12131758,2017-10-18,852.893616,2018-02-18,1215351,4.0,5000.0,8.0,852.893616,2018-02-18,6,234150,2016-08-01,0,32,8,8,8343,2018-07-09
8995849,2018-07-05,249.260101,2018-12-05,3919361,5.0,1460.0,8.3,249.260101,2018-12-05,6,623933,2017-08-01,1,33,26,4,4155,2019-02-27
3554193,2017-09-06,946.639893,2018-03-06,915087,6.0,5540.0,8.6,946.639893,2018-03-06,6,102853,2016-01-01,1,27,2,17,17096,2018-03-19


In [0]:
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20711267 entries, 0 to 20711266
Data columns (total 18 columns):
auditing_date    datetime64[ns]
due_amt          float32
due_date         datetime64[ns]
listing_id       uint32
order_id         float32
principal        float32
rate             float32
repay_amt        float32
repay_date       datetime64[ns]
term             uint32
user_id          uint32
reg_mon          datetime64[ns]
gender           uint32
age              uint32
cell_province    uint32
id_province      uint32
id_city          uint32
insertdate       datetime64[ns]
dtypes: datetime64[ns](5), float32(5), uint32(8)
memory usage: 1.9 GB


In [0]:

user_taglist.rename(columns={'insertdate': 'tag_insert_date'}, inplace=True)
user_taglist = user_taglist.sort_values(by='tag_insert_date', ascending=False).drop_duplicates('user_id').reset_index(drop=True)


In [0]:
train=pd.merge(train,user_taglist,on=['user_id'],how='left')

In [0]:
del user_taglist
gc.collect()

42

In [0]:
train.to_csv('train_new.csv',index=False)

In [0]:
train.sample(10)

,auditing_date,due_amt,due_date,listing_id,order_id,principal,rate,repay_amt,repay_date,term,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate,tag_insert_date,top1,top2,top3,top4,top5,top10
3679686,2018-02-22,155.226395,2018-03-22,2389688,1.0,910.0,8.0,155.226395,2018-03-22,6,700474,2017-10-01,1,27,29,29,29091,2018-05-12,2018-05-12,18.0,166.0,687.0,1898.0,4308.0,38533.0
14837843,2017-05-22,102.481003,2018-03-22,317364,10.0,1170.0,9.3,102.481003,2018-03-20,12,510335,2017-05-01,1,25,20,6,6281,2018-08-21,2018-08-21,18.0,166.0,687.0,1898.0,4308.0,37875.0
15142666,2017-12-20,114.471001,2018-01-20,1777977,1.0,1000.0,7.2,114.471001,2018-01-20,9,357423,2016-12-01,0,32,16,16,16146,2018-01-26,2018-09-24,18.0,166.0,687.0,1898.0,4308.0,38443.0
15960368,2018-04-26,102.102203,2018-08-26,3178847,4.0,1170.0,8.6,102.102203,2018-08-14,12,739837,2017-11-01,1,25,26,26,26057,2019-02-01,2019-02-01,18.0,166.0,687.0,1898.0,4308.0,38443.0
15670494,2017-12-03,190.335205,2018-06-03,1628798,6.0,1660.0,7.6,190.335205,2018-04-27,9,413297,2017-02-01,1,32,28,28,28340,2018-07-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN
8347123,2018-02-23,327.511200,2018-03-23,2395975,1.0,1920.0,8.0,327.511200,2018-03-15,6,718909,2017-11-01,1,28,4,4,4152,2018-06-06,NaT,NaN,NaN,NaN,NaN,NaN,NaN
5181382,2018-01-05,144.656403,2018-02-05,1914562,1.0,850.0,7.2,144.656403,2018-01-06,6,525053,2017-05-01,1,25,17,17,17242,2018-03-15,2018-10-08,18.0,166.0,687.0,1898.0,4308.0,43781.0
14591,2018-12-15,942.004089,2019-01-15,5127209,NaN,5540.0,6.9,942.004089,2019-01-15,6,9099,2014-08-01,1,28,26,26,26131,2018-12-14,2018-12-14,18.0,166.0,687.0,1894.0,4238.0,22312.0
20259857,2017-12-07,97.300301,2018-05-07,1663522,5.0,850.0,7.2,97.300301,2018-01-11,9,418130,2017-03-01,1,25,5,5,5021,2018-06-27,2018-08-16,18.0,166.0,687.0,1898.0,4308.0,41759.0
14039543,2018-01-23,884.957275,2018-06-23,2088522,5.0,5200.0,7.2,884.957275,2018-06-16,6,473343,2017-04-01,1,48,17,17,17290,2018-08-29,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# user_taglist.rename(columns={'insertdate': 'tag_insert_date'}, inplace=True)
# user_taglist = user_taglist.sort_values(by='tag_insert_date', ascending=False).drop_duplicates('user_id').reset_index(drop=True)


In [0]:
user_behavior_logs.sample(10)

,user_id,behavior_time,behavior_type
5550251,142601,2018-02-05 17:41:52,1
41413923,868500,2018-10-21 21:02:45,1
28869041,165760,2018-11-25 16:04:36,1
28530411,8611,2017-10-20 18:14:33,3
14039644,462882,2018-07-14 13:25:55,1
47894960,647816,2018-07-15 13:57:11,1
18546483,531671,2017-11-29 18:29:31,1
17664087,628204,2018-03-17 11:13:21,1
11757713,869755,2018-06-15 20:01:04,3
26668441,695878,2018-08-20 18:28:46,3


In [0]:
user_behavior_logs_times=user_behavior_logs.groupby('behavior_time').count()

In [0]:

user_behavior_logs_times.reset_index().rename({'user_id':'user_id_times'}).sample(10)


In [0]:
#################################################################################

In [0]:
all_data[all_data['repay_amt'].isnull()].shape

(130000, 7)

In [0]:
all_data.head()

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id
0,2018-04-25,72.1167,2018-05-25,3163926,72.1167,2018-05-25,748147
1,2018-06-09,258.7045,2018-07-09,3698760,258.7045,2018-07-08,672952
2,2018-02-18,307.9270,2018-03-18,2355665,\N,\N,404196
3,2018-01-13,252.9809,2018-02-13,1994522,252.9809,2018-02-13,342769
4,2018-06-01,107.6503,2018-07-01,3602352,107.6503,2018-06-25,828139


In [0]:
import gc
del listing_info['user_id'], listing_info['auditing_date']
gc.collect()

14

In [0]:
all_data=pd.merge(all_data,listing_info,on=['listing_id'],how='left')##只将all_data中的listing_id添加了一些维度，对于新增的listing_id并没有添加

In [0]:
all_data.sample(20)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id,term,rate,principal
112683,2018-11-19,121.3008,2018-12-19,4943841,121.3008,2018-12-18,833995,12,8.6,1390
769391,2018-11-21,326.3780,2018-12-21,4957486,326.378,2018-12-21,80274,12,8.6,3740
843706,2018-05-17,302.4463,2018-06-17,3427903,302.4463,2018-06-15,342946,6,8.6,1770
147577,2018-11-13,299.3252,2018-12-13,4904765,299.3252,2018-12-13,11206,12,8.6,3430
737902,2018-08-15,245.8456,2018-09-15,4209031,245.8456,2018-09-14,806539,6,8.3,1440
1066681,2019-03-09,383.0054,2019-04-09,5425226,NaN,NaN,564547,9,6.9,3350
534145,2018-03-19,1277.4966,2018-04-19,2688511,1277.4966,2018-03-26,234760,9,7.2,11160
962549,2018-09-25,478.0331,2018-10-25,4526515,478.0331,2018-10-25,139634,6,8.3,2800
719183,2018-02-25,394.0369,2018-03-25,2415975,394.0369,2018-03-19,627467,6,8.0,2310
139997,2018-04-26,1891.6457,2018-05-26,3179597,1891.6457,2018-05-26,160883,6,8.3,11080


In [0]:
all_data.shape

(1130000, 10)

In [0]:
user_info.head()

,user_id,reg_mon,gender,age,cell_province,id_province,id_city,insertdate
0,483833,2017-04,男,19,c29,c26,c26241,2018-12-11
1,156772,2016-05,男,31,c11,c11,c11159,2018-02-13
2,173388,2016-05,男,34,c02,c02,c02182,2018-08-21
3,199107,2016-07,女,25,c09,c09,c09046,2018-06-05
4,122560,2016-03,男,23,c05,c05,c05193,2018-04-02


In [0]:
## 表中有少数user不止一条记录，因此按日期排序，去重，只保留最新的一条记录。
user_info = user_info.sort_values(by='insertdate', ascending=False).drop_duplicates('user_id').reset_index(drop=True)

In [0]:
all_data=pd.merge(all_data,user_info,on=['user_id'],how='left')

In [0]:
all_data.sample(10)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id,term,rate,principal,reg_mon,gender,age,cell_province,id_province,id_city,insertdate
885066,2018-10-08,945.8228,2018-11-08,4612184,945.8228,2018-11-07,116374,6,8.3,5540,2016-02,女,24,c04,c21,c21299,2018-10-07
323,2018-06-21,131.6416,2018-07-21,3790584,131.6416,2018-07-21,719769,9,7.2,1150,2017-11,女,42,c24,c24,c24044,2018-06-20
393581,2018-09-08,72.4314,2018-10-08,4391798,72.4314,2018-09-21,268513,12,8.6,830,2016-09,女,34,c16,c16,c16146,2018-02-26
436420,2018-03-15,525.3824,2018-04-15,2641871,525.3824,2018-04-13,65580,6,8.0,3080,2015-10,男,27,c31,c31,c31114,2018-02-05
535421,2018-10-16,394.6893,2018-11-16,4668817,394.6893,2018-11-16,745574,3,7.2,1170,2017-12,女,25,c05,c05,c05021,2018-10-15
756932,2018-10-18,571.4387,2018-11-18,4683432,571.4387,2018-11-18,284529,6,8.0,3350,2016-10,男,25,c04,c04,c04331,2018-10-17
650945,2018-10-22,327.2507,2018-11-22,4714983,327.2507,2018-11-22,666661,12,8.6,3750,2017-09,男,26,c04,c04,c04264,2018-10-21
469354,2018-03-11,133.9311,2018-04-11,2593135,133.9311,2018-04-11,209081,9,7.2,1170,2016-07,男,26,c09,c09,c09167,2018-03-10
169179,2018-05-01,452.1605,2018-06-01,3233494,\N,\N,323739,9,7.2,3950,2016-11,男,32,c22,c22,c22308,2018-04-30
796803,2018-04-14,766.5604,2018-05-14,3029515,766.5604,2018-05-13,482494,6,8.3,4490,2017-04,男,27,c05,c05,c05296,2018-04-13


In [0]:
user_taglist.rename(columns={'insertdate': 'tag_insert_date'}, inplace=True)
user_taglist = user_taglist.sort_values(by='tag_insert_date', ascending=False).drop_duplicates('user_id').reset_index(drop=True)


In [0]:
all_data=pd.merge(all_data,user_taglist,on=['user_id'],how='left')

In [0]:
all_data.sample(10)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id,term,rate,principal,reg_mon,gender,age,cell_province,id_province,id_city,insertdate,taglist,tag_insert_date
1022751,2019-02-15,202.6137,2019-03-15,5336367,NaN,NaN,88883,9,7.2,1770,2015-12,男,31,c31,c31,c31255,2019-02-14,5906|719|5865|3369|4453|3006|70|4524|5723|4787...,2019-02-14
353326,2018-12-12,384.2833,2019-01-12,5107265,384.2833,2019-01-12,54469,6,6.9,2260,2015-09,女,34,c20,c19,c19239,2018-12-11,NaN,NaN
140123,2018-03-25,161.4041,2018-04-25,2757432,161.4041,2018-04-24,482680,9,7.2,1410,2017-04,男,37,c19,c19,c19104,2018-03-24,2710|1228|1541|2901|1402|2500|877|4592|1804|84...,2018-03-24
316259,2018-03-06,171.7065,2018-04-06,2536224,171.7065,2018-04-03,545759,9,7.2,1500,2017-06,女,35,c04,c04,c04344,2018-03-05,3017|1832|1204|1368|354|3695|3691|2648|1911|27...,2018-03-05
832785,2018-04-01,371.8615,2018-05-01,2848283,371.8615,2018-04-19,539922,6,8.0,2180,2017-06,男,24,c05,c05,c05296,2018-03-31,230|2242|1478|2164|2250|3324|1876|3781|1059|83...,2018-03-31
278046,2018-04-01,136.2205,2018-05-01,2845894,136.2205,2018-04-28,316148,9,7.2,1190,2016-11,男,22,c26,c26,c26241,2018-03-31,NaN,NaN
619391,2018-02-07,352.5707,2018-03-07,2242111,352.5707,2018-03-06,693139,9,7.2,3080,2017-10,男,20,c12,c12,c12007,2018-02-06,2134|3088|2253|4864|4749|1541|5320|719|2017|20...,2018-02-06
1070322,2019-03-20,198.7159,2019-04-20,5457877,NaN,NaN,686712,6,6.5,1170,2017-10,男,22,c23,c26,c26047,2019-03-19,60|2580|127|2710|3151|4219|2642|155|1911|4563|...,2019-03-19
1114162,2019-02-08,176.8383,2019-03-08,5311810,NaN,NaN,484890,6,6.9,1040,2017-04,女,27,c04,c04,c04251,2019-02-07,NaN,NaN
827317,2018-03-29,144.9917,2018-04-29,2814899,\N,\N,479372,6,8.0,850,2017-04,男,22,c21,c21,c21240,2018-03-28,2245|2431|943|1285|2649|1964|3580|124|1567|620...,2018-03-28


In [0]:
all_data=pd.merge(all_data,user_repay_logs[['listing_id','order_id']],on=['listing_id'],how='left')

In [0]:
all_data.sample(5)

,auditing_date,due_amt,due_date,listing_id,repay_amt,repay_date,user_id,term,rate,principal,reg_mon,gender,age,cell_province,id_province,id_city,insertdate,taglist,tag_insert_date,order_id
791713,2018-03-07,97.3003,2018-04-07,2546932,\N,\N,609863,9,7.2,850,2017-08,男,21,c29,c29,c29091,2018-03-06,3527|1826|2739|121|620|2431|1950|1823|676|1876...,2018-03-06,NaN
263385,2018-04-19,861.9668,2018-05-19,3090846,861.9668,2018-05-17,624811,9,7.2,7530,2017-08,男,26,c19,c19,c19335,2018-04-18,NaN,NaN,5.0
1449235,2018-04-23,2029.5713,2018-05-23,3144913,\N,\N,216253,9,7.2,17730,2016-07,男,31,c01,c29,c29128,2018-04-22,943|2794|645|449|445|2132|675|4224|3508|3972|4...,2018-11-25,2.0
109036,2018-11-03,124.7915,2018-12-03,4829996,124.7915,2018-12-01,655643,12,8.6,1430,2017-09,男,28,c02,c02,c02163,2018-03-04,1803|2580|1402|1029|5071|4770|1259|1787|2631|1...,2018-11-02,NaN
778126,2018-03-17,227.4265,2018-04-17,2662575,227.4265,2018-04-17,191284,12,7.6,2620,2016-06,女,25,c23,c23,c23141,2018-03-16,NaN,NaN,5.0


In [0]:
all_data.to_csv('all_data.csv',index=False)

In [0]:
train.loc[(train['auditing_date']>=pd.to_datetime('2018-01-05'))&(train['auditing_date']<=pd.to_datetime('2018-08-05'))].head()

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
0,748147,3163926,2018-04-25,2018-05-25,72.1167,2018-05-25,72.1167
1,672952,3698760,2018-06-09,2018-07-09,258.7045,2018-07-08,258.7045
2,404196,2355665,2018-02-18,2018-03-18,307.9270,\N,\N
3,342769,1994522,2018-01-13,2018-02-13,252.9809,2018-02-13,252.9809
4,828139,3602352,2018-06-01,2018-07-01,107.6503,2018-06-25,107.6503


In [0]:
print(train.describe())

              user_id    listing_id         due_amt
count  1000000.000000  1.000000e+06  1000000.000000
mean    446177.855141  3.797037e+06      452.026625
std     257534.686801  9.424460e+05      514.202155
min          1.000000  1.873193e+06       17.453300
25%     225120.750000  3.046201e+06      160.482400
50%     444321.000000  3.832456e+06      286.740100
75%     664508.250000  4.657050e+06      546.026800
max     924400.000000  5.207722e+06    18827.018400


In [0]:
print(listing_info.shape)

(5484891, 6)


In [0]:
print(listing_info['listing_id'].nunique())

5484891


In [0]:
# print(listing_info['user_id'].nunique())
# print((train['user_id'].append(test['user_id']).nunique())

set(listing_info['user_id'])-set(train['user_id'])-set(test['user_id'])

set()

In [0]:
train[train['user_id']==65747]	##train中的标的只是用户的部分标的情况，还有一些标的没有给出因为是抽样的把，但是listing_info中的标的有更多，包括不在train中的

,user_id,listing_id,auditing_date,due_date,due_amt,repay_date,repay_amt
84998,65747,1879801,2018-01-01,2018-02-01,437.2793,2018-01-23,437.2793


In [0]:
len(set(listing_info['listing_id'])-set(train['listing_id'])-set(test['listing_id']))
##4354891这么多的标的既不在train中也不在test中，但是若统计一段时间内用户的借款次数要把这些加上；预测的用户部分标的的还款时间
##也就是统计用户信息，以user_id为键，（或者user_id listing_id）

4354891

In [0]:
user_repay_logs[user_repay_logs['listing_id'].isin([1,2,3,4])]

,user_id,listing_id,order_id,due_date,due_amt,repay_date,repay_amt
234684,65747,2,12,2017-07-05,323.2095,2017-05-15,323.2095
2817251,133141,4,12,2017-07-06,55.1821,2017-01-13,55.1821
17182239,194014,1,12,2017-07-05,389.7784,2017-07-03,389.7784
17205590,4548,3,12,2017-07-06,973.1321,2017-07-06,973.1321


In [0]:
user_repay_logs[user_repay_logs['listing_id'].isin([5138741,3661187])]

,user_id,listing_id,order_id,due_date,due_amt,repay_date,repay_amt
10188421,3,3661187,4,2018-10-06,509.396,2018-10-06,509.396
10188422,3,3661187,5,2018-11-06,509.396,2018-11-06,509.396
10188423,3,3661187,6,2018-12-06,509.396,2018-12-06,509.396
10188424,3,3661187,2,2018-08-06,509.396,2018-08-06,509.396
10188425,3,3661187,1,2018-07-06,509.396,2018-07-06,509.396
10188426,3,3661187,3,2018-09-06,509.396,2200-01-01,509.396


In [0]:
print(train.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 7 columns):
user_id          1000000 non-null int64
listing_id       1000000 non-null int64
auditing_date    1000000 non-null datetime64[ns]
due_date         1000000 non-null datetime64[ns]
due_amt          1000000 non-null float64
repay_date       1000000 non-null object
repay_amt        1000000 non-null object
dtypes: datetime64[ns](2), float64(1), int64(2), object(2)
memory usage: 162.4 MB
None


In [0]:
print((train['due_date']-train['auditing_date']).dt.days.head())

NameError: ignored

In [0]:
train['user_id'].nunique()/train['user_id'].shape[0]

0.823732

In [0]:
print(train.shape)
print(test.shape)

(1000000, 7)
(130000, 5)


In [0]:
print(train['user_id'].nunique())

823732


In [0]:
print(train['listing_id'].nunique())##说明不同的用户，有相同的标的，

1000000


In [0]:
print(test['user_id'].nunique())


130000


In [0]:
sort_test=test.sort_values(by='user_id')

In [0]:
print(sort_test.head(10))

        user_id  listing_id auditing_date    due_date    due_amt
26177         5     5348580    2019-02-18  2019-03-18   205.7445
95146         8     5288930    2019-02-02  2019-03-02   521.9878
7011         20     5335952    2019-02-15  2019-03-15    88.1426
117274       22     5382105    2019-02-26  2019-03-26   125.7629
55784        24     5433181    2019-03-12  2019-04-12   560.4247
18638        29     5422932    2019-03-09  2019-04-09   467.6095
84268        35     5414862    2019-03-07  2019-04-07   226.9417
25043        41     5462399    2019-03-22  2019-04-22  1450.8678
34878        46     5480102    2019-03-29  2019-04-29   220.6568
35276        55     5424428    2019-03-09  2019-04-09   290.1735


In [0]:
print(sort_test.shape)

(130000, 5)


In [0]:
print(sort_test['user_id'].nunique())

130000


In [0]:
sort_info=listing_info.sort_values(by='listing_id')

In [0]:
print(sort_info.head())

         user_id  listing_id auditing_date  term  rate  principal
1341641   194014           1    2016-07-05    12   9.3       4450
1341640    65747           2    2016-07-05    12   9.3       3690
2715804     4548           3    2016-07-06    12   9.3      11110
1341639   133141           4    2016-07-06    12   9.3        630
4089125   198392           5    2016-07-07    12   9.3       2800


In [0]:
sort_info_user=listing_info.sort_values(by='user_id')

In [0]:
print(sort_info_user.head(10))#同一个用户在不同时间有不同的成交标的

         user_id  listing_id auditing_date  term  rate  principal
4307755        1     5138741    2018-12-17     9   7.2       5540
5016150        1      445999    2017-06-21    12   9.3       1380
2603034        2     1655550    2017-12-06    12   8.0       2800
1610929        2     4593188    2018-10-05     6   8.3       5430
623555         3      806012    2017-08-19    12   9.3       9740
928897         3     3661187    2018-06-06     9   7.2       4450
957421         3     1917365    2018-01-05    12   7.6       2800
50605          3     1353129    2017-11-03    12   8.6       8350
410070         3     4373926    2018-09-06     6   8.3       5320
898715         3     5079125    2018-12-08     9   7.2       5540


In [0]:
sort_train=train.sort_values(by='user_id')
#同一个人在不同时间有不同的成交标的##用户的成交次数作为特征
#借款用户可能有多个标的记录，但在测试集时间范围内每位用户只有一条记录。

In [0]:
print(sort_train.head(20))

        user_id  listing_id auditing_date    due_date    due_amt  repay_date  \
716258        1     5138741    2018-12-17  2019-01-17   634.1695  2019-01-11   
699724        2     4593188    2018-10-05  2018-11-05   927.0431  2018-11-04   
612838        3     3661187    2018-06-06  2018-07-06   509.3960  2018-07-06   
723079        3     5079125    2018-12-08  2019-01-08   634.1695  2019-01-08   
347041        4     1998686    2018-01-14  2018-02-14  1013.0686  2018-02-11   
5856          6     2896524    2018-04-04  2018-05-04   412.8005  2018-05-01   
81254         7     3737371    2018-06-13  2018-07-13  4036.6015  2018-07-13   
753040        8     4036140    2018-07-19  2018-08-19   283.4052          \N   
726131        9     4853712    2018-11-06  2018-12-06  1269.3415  2018-12-05   
887121        9     4777798    2018-10-28  2018-11-28   292.3439  2018-11-26   
150732       10     3225231    2018-04-30  2018-05-30   626.5648  2018-05-30   
946370       11     3380646    2018-05-1

In [0]:
sort_train_by_listid=train.sort_values(by='listing_id')

In [0]:
print(sort_train_by_listid.head(20))

        user_id  listing_id auditing_date    due_date   due_amt  repay_date  \
395543   438081     1873193    2018-01-01  2018-02-01   97.3003  2018-01-28   
864992   679097     1873199    2018-01-01  2018-02-01  200.8172  2018-01-04   
404472   633241     1873200    2018-01-01  2018-02-01  127.0628          \N   
271760   498924     1873205    2018-01-01  2018-02-01  226.6526  2018-02-01   
439912   342463     1873207    2018-01-01  2018-02-01  125.9181  2018-02-01   
466469   297792     1873212    2018-01-01  2018-02-01  163.3767  2018-01-31   
237865   711380     1873215    2018-01-01  2018-02-01  133.9311  2018-01-26   
639101   276089     1873220    2018-01-01  2018-02-01  267.8622  2018-01-30   
582993     8808     1873221    2018-01-01  2018-02-01  169.4171  2018-02-01   
393029   137332     1873223    2018-01-01  2018-02-01  116.7604  2018-01-30   
916595    84259     1873224    2018-01-01  2018-02-01  383.4779  2018-01-31   
475342   499298     1873232    2018-01-01  2018-02-0

In [0]:
sort_user_info=user_info.sort_values(by='user_id')

In [0]:
print(sort_user_info.head(10))#用户信息可能发生变更，表中同一用户可能存在多条数据  ##记录变更次数作为特征

        user_id  reg_mon gender  age cell_province id_province id_city  \
423710        1  2008-03      男   35           c22         c31  c31067   
468220        2  2008-03      男   46           c09         c09  c09205   
245236        3  2008-03      男   35           c22         c05  c05126   
233132        4  2008-04      男   36           c17         c17  c17246   
953223        5  2008-04      男   45           c04         c27  c27120   
611348        6  2008-04      男   36           c14         c14  c14027   
783710        7  2008-04      男   38           c05         c05  c05021   
241066        8  2008-04      男   36           c04         c04  c04152   
243463        8  2008-04      男   35           c04         c04  c04152   
815086        9  2008-04      男   34           c05         c05  c05169   

        insertdate  
423710  2018-12-16  
468220  2018-10-04  
245236  2018-06-05  
233132  2018-01-13  
953223  2019-02-17  
611348  2018-04-03  
783710  2018-06-12  
241066  2019-02-0

In [0]:
sort_user_taglist=user_taglist.sort_values(by='user_id')

In [0]:
print(sort_user_taglist.head(20))#用户标签可能发生变更，表中同一用户可能存在多条数据；若在表中无法查到用户标签信息，则表示该用户标签信息未知。

        user_id                                            taglist  insertdate
88108         2  2017|4853|3345|1401|320|727|3703|554|1804|2139...  2018-10-04
321635        4  323|683|2250|436|1288|1044|1876|181|357|50|365...  2018-01-13
221336        5  1868|2580|483|1303|904|3604|4028|2049|196|3551...  2019-02-17
133862        7  2005|271|2365|2216|933|1804|1788|719|2572|469|...  2018-06-12
502184        9  2710|1911|3504|3574|864|4688|127|2642|2127|420...  2018-10-27
501338        9  5852|3703|287|5762|5539|5465|4207|3574|3519|11...  2018-11-05
386047       10  1183|2492|2049|472|176|592|4703|266|595|721|31...  2018-04-29
429398       12  2017|1804|883|631|2365|552|1178|970|1952|2436|...  2018-03-28
273693       13  2809|638|1820|550|20|271|4938|2026|1697|1395|1...  2018-03-04
570753       15  2047|141|1863|325|4185|58|1127|4852|4766|620|4...  2018-03-03
330097       17  411|1255|5539|287|3017|1280|127|305|418|3519|1...  2018-11-27
116627       18  2018|1729|3844|2443|1979|3219|1769|

In [0]:
sort_user_behavior_logs=user_behavior_logs.sort_values(by='user_id')

In [0]:
print(sort_user_behavior_logs.shape)

(55781271, 3)


In [0]:
print(sort_user_behavior_logs['user_id'].nunique())

920161


In [0]:
print(sort_user_behavior_logs.head(20))#相同用户的相同行为在同一秒内可能出现多条数据。###这样的数据要去重

          user_id        behavior_time  behavior_type
36614258        1  2018-08-11 20:37:46              1
36535106        1  2018-07-04 17:29:50              3
36535100        1  2018-07-04 17:29:47              3
36535096        1  2018-07-04 17:29:47              3
36535094        1  2018-07-04 17:29:47              3
36535093        1  2018-07-04 17:29:47              3
36535092        1  2018-07-04 17:29:50              3
36535088        1  2018-07-04 17:30:44              3
36535085        1  2018-07-04 17:29:47              3
36535083        1  2018-07-04 17:29:47              3
36535107        1  2018-07-04 17:29:50              3
36535082        1  2018-07-04 17:29:47              3
36535075        1  2018-07-04 17:29:47              3
36535074        1  2018-07-04 17:29:47              3
36520284        1  2018-08-09 16:09:10              1
36406585        1  2018-09-27 12:55:38              1
36406838        1  2018-09-27 12:55:44              1
36523169        1  2018-07-0

In [0]:
train_df = pd.read_csv(path+'train.csv', parse_dates=['auditing_date', 'due_date', 'repay_date'])



In [0]:
train_df['repay_date'] = train_df[['due_date', 'repay_date']].apply(
    lambda x: x['repay_date'] if x['repay_date'] != '\\N' else x['due_date'], axis=1
)

In [0]:
train_df['repay_amt'] = train_df['repay_amt'].apply(lambda x: x if x != '\\N' else 0).astype('float32')
train_df['label'] = (train_df['repay_date'] - train_df['auditing_date']).dt.days


In [0]:
print(train_df[train_df['repay_amt']==0].head())#此时就是刚才定义的，未还款时将应还款时间赋值成实际还款时间了

    user_id  listing_id auditing_date   due_date   due_amt repay_date  \
2    404196     2355665    2018-02-18 2018-03-18  307.9270 2018-03-18   
9    757631     4509276    2018-09-23 2018-10-23  394.3774 2018-10-23   
20   279960     3220453    2018-04-30 2018-05-30  320.5188 2018-05-30   
27   175815     2602604    2018-03-12 2018-04-12  486.1024 2018-04-12   
44   519357     4511127    2018-09-23 2018-10-23  199.7496 2018-10-23   

    repay_amt  label  
2         0.0     28  
9         0.0     30  
20        0.0     30  
27        0.0     31  
44        0.0     30  


In [0]:
train_df.loc[train_df['repay_amt'] == 0, 'label'] = 32

In [0]:
print(train_df[train_df['label']==32].head())

    user_id  listing_id auditing_date   due_date   due_amt repay_date  \
2    404196     2355665    2018-02-18 2018-03-18  307.9270 2018-03-18   
9    757631     4509276    2018-09-23 2018-10-23  394.3774 2018-10-23   
20   279960     3220453    2018-04-30 2018-05-30  320.5188 2018-05-30   
27   175815     2602604    2018-03-12 2018-04-12  486.1024 2018-04-12   
44   519357     4511127    2018-09-23 2018-10-23  199.7496 2018-10-23   

    repay_amt  label  
2         0.0     32  
9         0.0     32  
20        0.0     32  
27        0.0     32  
44        0.0     32  


In [0]:
import gc
clf_labels = train_df['label'].values
amt_labels = train_df['repay_amt'].values
del train_df['label'], train_df['repay_amt'], train_df['repay_date']
gc.collect()
train_due_amt_df = train_df[['due_amt']]
train_num = train_df.shape[0]

In [0]:
test_df = pd.read_csv(path+'test.csv', parse_dates=['auditing_date', 'due_date'])
sub = test_df[['listing_id', 'auditing_date', 'due_amt']]
df = pd.concat([train_df, test_df], axis=0, ignore_index=True)


In [0]:
df.sample(10)

,user_id,listing_id,auditing_date,due_date,due_amt
861181,778706,4521876,2018-09-25,2018-10-25,447.3024
291831,114226,3366801,2018-05-11,2018-06-11,269.9802
673289,185872,2407641,2018-02-24,2018-03-24,340.2717
767063,210427,2772029,2018-03-26,2018-04-26,192.3113
71410,50760,3083894,2018-04-19,2018-05-19,1337.0216
648985,530469,4023219,2018-07-18,2018-08-18,634.1695
409724,510715,1934316,2018-01-07,2018-02-07,299.9140
778986,5119,3477764,2018-05-21,2018-06-21,144.2334
406892,634327,2085971,2018-01-23,2018-02-23,97.3003
877315,273347,3011605,2018-04-13,2018-05-13,385.8409


001_规则判断记录.ipynb          paipaidai_rule.py
1.baseline.py                   拍拍贷常见问题.doc
GBDT.py                         挺无聊的一个代码lb_8907.163500.py
MagicMirror_baseline_33分类.py  简单的EDA及规则.ipynb


In [0]:
from lightgbm.sklearn import LGBMClassifier


In [0]:
clf = LGBMClassifier(
    learning_rate=0.05,
    n_estimators=10000,
    subsample=0.8,
    subsample_freq=1,
    colsample_bytree=0.8,
    random_state=2019,
     device='gpu',
)

In [0]:
trn_x=[[1,2,3],[2,3,4]]
trn_y=[0,1]

In [0]:
clf.fit(
        trn_x, trn_y, 
        eval_set=[(trn_x, trn_y), (trn_x, trn_y)],
        early_stopping_rounds=100, verbose=5
    )

LightGBMError: ignored

In [0]:
####eda
print(train.head())


   user_id  listing_id auditing_date   due_date   due_amt  repay_date  \
0   748147     3163926    2018-04-25 2018-05-25   72.1167  2018-05-25   
1   672952     3698760    2018-06-09 2018-07-09  258.7045  2018-07-08   
2   404196     2355665    2018-02-18 2018-03-18  307.9270          \N   
3   342769     1994522    2018-01-13 2018-02-13  252.9809  2018-02-13   
4   828139     3602352    2018-06-01 2018-07-01  107.6503  2018-06-25   

  repay_amt  
0   72.1167  
1  258.7045  
2        \N  
3  252.9809  
4  107.6503  


In [0]:
print(train.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 7 columns):
user_id          1000000 non-null int64
listing_id       1000000 non-null int64
auditing_date    1000000 non-null datetime64[ns]
due_date         1000000 non-null datetime64[ns]
due_amt          1000000 non-null float64
repay_date       1000000 non-null object
repay_amt        1000000 non-null object
dtypes: datetime64[ns](2), float64(1), int64(2), object(2)
memory usage: 162.4 MB
None


In [0]:
print(train.describe())

              user_id    listing_id         due_amt
count  1000000.000000  1.000000e+06  1000000.000000
mean    446177.855141  3.797037e+06      451.799866
std     257534.686801  9.424460e+05      514.070251
min          1.000000  1.873193e+06       17.453300
25%     225120.750000  3.046201e+06      160.482407
50%     444321.000000  3.832456e+06      286.740112
75%     664508.250000  4.657050e+06      546.026794
max     924400.000000  5.207722e+06    18827.017578


In [0]:
##更改数据类型节省变量


for each in ['user_id','listing_id']:
  train[each]=train[each].astype('int32')##整型变量比字符串变量节省内存
for each in ['due_amt']:
  train[each]=train[each].astype('float32')


In [0]:
train.shape

(1000000, 7)

In [0]:
train[train['repay_date']=='\\N'].count()

user_id          117192
listing_id       117192
auditing_date    117192
due_date         117192
due_amt          117192
repay_date       117192
repay_amt        117192
dtype: int64

In [0]:
print(117192/1000000)##只有11.72%的标的没有还款，大部分标的都按时还款

0.117192


In [0]:
print(train[train['repay_date']=='\\N']['user_id'].value_counts().sort_values())

133893    0
850134    0
850132    0
850131    0
850130    0
850128    0
850127    0
850126    0
850125    0
850123    0
850122    0
850121    0
850120    0
850119    0
850080    0
850077    0
850264    0
850033    0
850046    0
850045    0
850044    0
850043    0
850135    0
850042    0
850136    0
850138    0
850105    0
850104    0
850103    0
850102    0
         ..
241016    3
189589    3
643672    3
672703    3
696998    3
331230    3
284355    3
264216    3
371498    3
262370    3
551256    3
434748    3
434693    3
86824     3
627973    3
129377    3
44324     3
48142     3
741229    3
83544     3
623101    3
804774    3
593585    4
319907    4
260477    4
259733    4
794221    4
434987    4
271000    4
571576    4
Name: user_id, Length: 823732, dtype: int64


In [0]:
print(train[train['user_id']==241016])

       user_id listing_id auditing_date   due_date     due_amt repay_date  \
274761  241016    4422293    2018-09-12 2018-10-12  833.143616         \N   
501054  241016    3989005    2018-07-13 2018-08-13  677.782715         \N   
610754  241016    4580505    2018-10-03 2018-11-03  332.915985         \N   

       repay_amt  
274761        \N  
501054        \N  
610754        \N  


In [0]:
print(train[train['user_id'].isin([571576,804774])].sort_values(by=['user_id','auditing_date']))

##对于某一个标的，一共被多少人借过，有多少人没还(这些人的特征是什么)
##根据user_id统计人的特征，然join得到相应标的上人特征### 人 时间 标的；例如：一共借了多少次（分全量数据，和滑窗数据特征），每次借的金额处于哪个等级，（需要多借款金额离散化）逾期了多少次

       user_id listing_id auditing_date   due_date     due_amt  repay_date  \
218310  571576    2508866    2018-03-04 2018-04-04   97.300301          \N   
40171   571576    3359022    2018-05-11 2018-06-11  133.931107          \N   
726744  571576    5138561    2018-12-17 2019-01-17  102.102203          \N   
472117  571576    5163044    2018-12-21 2019-01-21  101.344101          \N   
222879  804774    4029352    2018-07-18 2018-08-18  426.815186          \N   
175242  804774    4223087    2018-08-17 2018-09-17  223.651199  2018-09-17   
100999  804774    4623660    2018-10-10 2018-11-10  630.828186          \N   
9160    804774    4776702    2018-10-28 2018-11-28  232.765503          \N   

       repay_amt  
218310        \N  
40171         \N  
726744        \N  
472117        \N  
222879        \N  
175242  223.6512  
100999        \N  
9160          \N  


In [0]:
print(train[train['listing_id']==2508866])

       user_id listing_id auditing_date   due_date    due_amt repay_date  \
218310  571576    2508866    2018-03-04 2018-04-04  97.300301         \N   

       repay_amt  
218310        \N  


In [0]:
print(train.sort_values())